In [419]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import psycopg2
import os

In [420]:
def conectar_bd():
    conn = psycopg2.connect(
        host=os.getenv("DB_HOST"),
        database=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        port=os.getenv("DB_PORT")
    )
    return conn, conn.cursor()

In [421]:
conectar_bd()

(<connection object at 0x000002C265B6C7B0; dsn: 'user=postgres password=xxx dbname=Proyecto_Sephora host=localhost port=5432', closed: 0>,
 <cursor object at 0x000002C2699A7760; closed: 0>)

## Análisis General Sephora

Se parte de una query que trae los aspectos más relevantes de los productos, es decir, lo que les identifica (la url), también el identificador único seriado (empleado en el análisis exploratorio para los outliers), el nombre, su descripción, la marca, la categoría y subcategoría a la que pertenecen, el éxito que tienen (a partir del numero de valoraciones y la valoración) y el numero de variaciones de producto. Este análisis sirve para obtener una primera aproximación sobre la tienda de Sephora y poder comenzar a posicionar a Sephora Collection con respecto al resto de las marcas. 

In [422]:
def ejecutar_query(query):
    conn, cur = conectar_bd()
    cur.execute(query)
    cols = [desc[0] for desc in cur.description]
    datos = cur.fetchall()
    cur.close()
    conn.close()
    return pd.DataFrame(datos, columns=cols)

In [423]:
query_general = """
select 
    p.url_producto,
    p.id_producto,
    p.nombre,
    p.descripcion,
    m.nombre_marca, 
    c.nombre_categoria,
    s.nombre_subcategoria,
    h.precio,
    h.numero_valoraciones,
    h.valoracion,
    h.numero_variaciones
from productos p 
join marcas m on p.id_marca = m.id_marca
left join categorias c on p.id_categoria = c.id_categoria
left join subcategorias s on p.id_subcategoria = s.id_subcategoria
join historico h on p.id_producto = h.id_producto;
"""

In [424]:
df = ejecutar_query(query_general)
df.head()

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
0,https://www.sephora.es/p/soft-lit-naturally-lu...,1,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7,50
1,https://www.sephora.es/p/triclone-skin-tech-fo...,2,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5,51
2,https://www.sephora.es/p/triclone-skin-tech-hy...,3,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1,30
3,https://www.sephora.es/p/color-fuse-talc-free-...,4,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8,6
4,https://www.sephora.es/p/bio-radiant-gel-powde...,5,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8,5


In [425]:
df.shape

(1829, 11)

De primeras se cuenta con las 11 columnas ya previamente mencionadas y 1829 filas, es decir, 1829 productos de maquillaje en todo Sephora.

In [426]:
def info_reporte(dataframe):
    """
    Genera un informe sobre los valores nulos y tipos de datos de un DataFrame.

    Args:
        dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
        pd.DataFrame: Un DataFrame con el número y porcentaje de valores nulos por columna,
                      junto con el tipo de dato de cada columna.
    """
    df_report = pd.DataFrame()
    df_report["Numero_nulos"] = dataframe.isnull().sum()
    df_report["Porcentaje_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0]*100), 2)
    df_report["Tipo_dato"] = dataframe.dtypes
    return df_report

In [427]:
info_reporte(df)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
url_producto,0,0.00,object
id_producto,0,0.00,int64
nombre,3,0.16,object
descripcion,98,5.36,object
nombre_marca,0,0.00,object
nombre_categoria,9,0.49,object
nombre_subcategoria,33,1.80,object
precio,0,0.00,float64
numero_valoraciones,0,0.00,int64
valoracion,0,0.00,float64


Con respecto a los valores nulos, encontramos 3 productos con nulos en el nombre, 98 con nulos en la descripción, 9 con nulos en la categoría y 33 con nulos en la subcategoría. No es necesario tratar estos nulos ya que tienen que estar ahí, pues correctamente hay determinados productos de los cuales se desconoce su nombre, su descripción, su categoría o su subcategoría. De todas formas, el % de nulos que hay en cada una de estas columnas es mínimo por lo que estos productos se van a mantener ya que el resto de información puede resultar muy relevante para el análisis. Por otro lado, en cuanto al tipo de dato, en la extracción hay un proceso de transformación asegurando que cada columna se extraiga con el tipo de dato correcto y por lo tanto, todas las columnas tienen el tipo de dato correcto.

In [428]:
df["url_producto"].duplicated().sum()

np.int64(7)

In [429]:
df.drop_duplicates(subset=["url_producto"], inplace=True)
df.shape

(1822, 11)

En cuanto a los duplicados, no debería haber ningún duplicado por el campo de url del producto, ya que este campo es el identificador único de los productos. No debería haberlos porque al haber realizado únicamente un scrapeo (el primero), los productos no deben aparecer más de una vez en la tabla de histórico. Por ahora no se ha logrado identificar en el código donde puede estar el punto donde se comete este error generando duplicados sobre 7 productos (identifica como que los productos ya están en la tabla "productos" y por lo tanto inserta una nueva fila a la tabla "historico", convirtiéndose en un duplicado erróneo). De momento, y al ser muy pocos los duplicados, para poder realizar un análisis con los datos correctamente recogidos, elimino directamente las filas duplicadas y por lo tanto nos quedamos con el número correcto de productos de maquillaje en toda Sephora, que son 1822. 

### Número total de productos en Sephora

In [430]:
numero_total_productos = df.shape[0]
print(f"Numero total de productos: {numero_total_productos}")

Numero total de productos: 1822


In [431]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,1822.0,911.500000,526.110413,1.00,456.25,911.50,1366.75,1822.0
precio,1822.0,37.201850,18.895532,3.99,25.00,34.99,45.00,169.0
numero_valoraciones,1822.0,590.726674,1707.459476,0.00,3.00,42.50,406.75,20035.0
valoracion,1822.0,3.799671,1.543679,0.00,3.90,4.40,4.70,5.0
numero_variaciones,1822.0,4.249177,7.088864,0.00,0.00,2.00,6.00,60.0


Descriptivos de las variables numéricas: 
- Precio: El precio de los productos en Sephora oscila entre 3,99 € y 169 €. La media se sitúa en 37,20 €, mientras que la mediana es de 34,99 €, lo que indica que la distribución está sesgada a la derecha. Esto significa que hay una mayoría de productos con precios moderados, pero algunos artículos significativamente más caros elevan el promedio. El 25% de los productos cuesta menos de 25 €, el 50% menos de 34,99 €, y el 75% menos de 45 €. Solo el 25% de los productos supera los 45 €, y un pequeño número alcanza precios muy altos, lo que sugiere la existencia de productos premium que pueden considerarse outliers dentro de la gama de precios.
- Número de valoraciones: La cantidad de valoraciones por producto varía de forma muy amplia, desde 0 hasta 20.035. La media es de 590,73 valoraciones, pero la mediana es de solo 42,5, lo que refleja una distribución altamente sesgada a la derecha. Esto indica que la mayoría de los productos tiene muy pocas valoraciones, mientras que unos pocos concentran una gran cantidad, elevando considerablemente el promedio. Aproximadamente el 25% de los productos cuenta con menos de 3 valoraciones, la mitad tiene menos de 42,5, y el 75% no supera las 406,75 valoraciones. Solo una cuarta parte de los productos supera ese umbral, y algunos alcanzan cifras muy altas, evidenciando casos claramente atípicos. Esta distribución sugiere que la visibilidad o la popularidad está muy concentrada en unos pocos productos dentro del catálogo.
- Valoración: Las valoraciones de los productos se mueven entre 0 y 5 puntos. La media es de 3,80, mientras que la mediana se sitúa en 4,40, lo que revela una ligera asimetría hacia la izquierda. Esto sugiere que la mayoría de los productos recibe buenas puntuaciones, pero algunos obtienen valoraciones muy bajas, lo que reduce el promedio. Un cuarto de los productos tiene una valoración inferior a 3,9, mientras que el 50% no supera los 4,4 y el 75% se sitúa por debajo de 4,7. Solo una pequeña proporción logra valoraciones superiores a ese nivel, alcanzando incluso el máximo de 5. Por lo tanto, aunque las valoraciones son en general altas, existen productos con puntuaciones bajas que podrían indicar problemas de calidad o satisfacción del cliente, y que pueden considerarse outliers negativos.
- El número de variaciones disponibles por producto, como diferentes tonos o tamaños, varía desde 0 hasta 60. La media es de 4,25 variaciones por producto, mientras que la mediana es de 2, lo cual indica una distribución fuertemente sesgada a la derecha. Esto significa que la mayoría de los productos tiene pocas opciones, pero algunos ofrecen una amplia gama de variaciones, lo que eleva la media general. El 25% de los productos no tiene ninguna variación, la mitad tiene dos o menos, y el 75% no supera las seis variaciones. Solo una minoría supera este número, y unos pocos productos llegan a tener hasta 60, situándose claramente como outliers. Esta situación es habitual en productos como bases de maquillaje, pintalabios o correctores, donde se requiere una gran diversidad de tonos para adaptarse a diferentes perfiles de consumidores.

### Valores representativos de las variables numéricas

En este análisis, se ha optado por utilizar la mediana en lugar de la media como medida representativa de las variables numéricas (precio, número de valoraciones, valoración y número de variaciones). Esta decisión se debe a que los datos presentan distribuciones claramente sesgadas y la presencia de valores atípicos (outliers) que distorsionan significativamente la media. Por tanto, el uso de la mediana permite describir con mayor precisión el comportamiento típico de los productos en Sephora, evitando que unos pocos valores extremos condicionen el análisis. Es una elección más robusta y fiel a la realidad de la distribución general del catálogo.

In [432]:
mediana_precio = df["precio"].median()
print(f"Mediana precio: {mediana_precio}")
mediana_num_valoraciones = df["numero_valoraciones"].median()
print(f"Mediana numero valoraciones: {mediana_num_valoraciones}")
mediana_valoracion = df["valoracion"].median()
print(f"Mediana valoracion: {mediana_valoracion}")
mediana_num_variaciones = df["numero_variaciones"].median()
print(f"Mediana numero variaciones: {mediana_num_variaciones}")

Mediana precio: 34.99
Mediana numero valoraciones: 42.5
Mediana valoracion: 4.4
Mediana numero variaciones: 2.0


### Observación de outliers de variables numéricas

In [449]:
def detectar_outliers(df, columna, modo='superior'):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    umbral_inferior = Q1 - 1.5 * IQR
    umbral_superior = Q3 + 1.5 * IQR

    if modo == 'superior':
        outliers = df[df[columna] > umbral_superior]
    else:
        outliers = df[df[columna] < umbral_inferior]

    return outliers[['nombre', 'descripcion', 'nombre_marca', 'nombre_categoria', 'nombre_subcategoria', 'precio', 'numero_valoraciones', 'valoracion', 'numero_variaciones']]

df_con_valoraciones = df[df["valoracion"] != 0]

outliers_precio = detectar_outliers(df, 'precio', modo='superior')
outliers_numero_valoraciones = detectar_outliers(df, 'numero_valoraciones', modo='superior')
outliers_valoracion = detectar_outliers(df_con_valoraciones, 'valoracion', modo='inferior')
outliers_numero_variaciones = detectar_outliers(df, 'numero_variaciones', modo='superior')

In [440]:
outliers_precio.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
12,HD Skin All-In-One Palette,Paleta de maquillaje todo en uno,Make Up For Ever,Paletas maquillaje,Paleta rostro,85.0,263,4.0,2
15,Diorshow 5 Couleurs edición limitada cannage,Paleta de sombras de ojos,Dior,Ojos,Paleta de sombras de ojos,76.0,0,0.0,4
22,Invisible UV Flawless Poreless Primer,Base de maquillaje,Charlotte Tilbury,Rostro,Primer & spray fijador,79.0,3,4.7,0
101,Green Flash,Kit pro Hortencia & Poppy Red,Manucurist,Estuches maquillaje,Estuches de manicura,86.0,0,0.0,0
189,HD Skin Sculting Palette,Paleta Modeladora del rostro Crema,Make Up For Ever,Paletas maquillaje,Paleta rostro,84.0,179,4.6,0


In [441]:
outliers_precio.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,56.0,103.462321,23.587274,76.0,85.0,96.00,115.2475,169.0
numero_valoraciones,56.0,101.267857,282.616701,0.0,2.0,16.50,44.2500,1959.0
valoracion,56.0,3.800000,1.824978,0.0,4.0,4.65,4.9000,5.0
numero_variaciones,56.0,3.196429,5.955653,0.0,0.0,0.00,4.0000,30.0


In [442]:
outliers_precio.describe(include='object').T

,count,unique,top,freq
nombre,56,54,Green Flash,2
descripcion,51,39,Paleta de sombras de ojos,7
nombre_marca,56,15,Sisley,19
nombre_categoria,55,7,Rostro,27
nombre_subcategoria,55,16,Base de maquillaje,12


Los productos outliers por precio en Sephora tienen un precio significativamente más alto, con una mediana de 96.00, en comparación con los productos generales, cuya mediana es de 34.99. Además, estos productos también tienen mejores valoraciones, con una mediana de 4.65, superior a la mediana general de 4.40. En cuanto al número de valoraciones, los outliers tienen una mediana de 16.50, menor a la mediana de 42.50 de los productos generales. En términos de variaciones, la mediana de los outliers es de 0.00, indicando que estos productos tienen menos opciones disponibles en comparación con los productos generales, cuya mediana de variaciones es de 2.00. Estos productos provienen principalmente de marcas de lujo como Sisley y se concentran en las categorías de Rostro y Base de maquillaje. En resumen, los productos más caros de Sephora tienden a tener mejores valoraciones, menos valoraciones y menos opciones disponibles en comparación con los productos más económicos.

In [443]:
pio.renderers.default = "browser"

fig = px.box(df, y="precio", hover_data=["id_producto"])
fig.update_layout(title="Boxplot del precio con ID del producto")
fig.show()

In [444]:
df[df["id_producto"] == 1410]

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
1415,https://www.sephora.es/p/sisle%c3%bfa-le-teint...,1410,Sisleÿa le Teint,Base de maquillaje antiedad,Sisley,Rostro,Base de maquillaje,169.0,27,4.7,13


Dentro de los outliers por precio si que es verdad que hay un producto concreto que si destaca un poco como outlier con respecto al resto que es una base de maquillaje de la marca Sisley que recoge el precio máximo al que se vende en Sephora - 169 euros. Aún así dentro de los 56 productos que destacan como outliers de precio, ya se observana que las bases de maquillaje (en 12 productos), así como la marca Sisley (en 19) son de los productos que más había. Sin embargo, la marca Sisley no es la marca con el precio mediano más alto de la tienda, ya que es la segunda y La Mer la primera. Aun así, La Mer también presenta algunos productos que destacan como outliers en precio. Sin embargo, esto sugiere que, aunque Sisley tiene más productos con precios muy altos (incluido el más caro del análisis), también ofrece otros productos con precios más moderados, lo que reduce su mediana de precio. En cambio, La Mer parece mantener de forma más consistente un nivel de precios elevado en toda su gama, lo que explicaría que tenga la mediana de precios más alta entre todas las marcas.

Del mismo modo, algo similar ocurre con la subcategoría de bases de maquillaje. A pesar de ser una de las más presentes entre los productos con precios considerados outliers, su precio mediano no es el más alto. De hecho, está por detrás de otras subcategorías como las paletas de rostro, las paletas de sombras de ojos y las de contouring. Esto sugiere que, aunque existen bases de maquillaje con precios muy elevados, también hay una amplia oferta de productos en rangos más asequibles dentro de esta subcategoría, lo cual hace que su mediana no sea tan alta. En cambio, las subcategorías que sí presentan una mediana superior podrían estar más concentradas en productos de alta gama o formatos más costosos por defecto, como ocurre con muchas paletas multifunción.

In [445]:
outliers_numero_valoraciones.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
34,Vanish™ Airbrush Concealer,Corrector antiojeras,Hourglass,Rostro,Antiojeras & corrector,42.00,3247,4.3,29
43,Pout Preserve Lip Treatment,Cuidado labial hidratante y antiedad,Olehenriksen,Labios,Bálsamo labial,21.00,1185,4.4,8
54,Limtless Lash Mascara,Mascara,Ilia,Ojos,Máscara de pestañas,35.00,4117,4.5,2
76,Puffer Gloss,"Brillo de labios extra volumen, extra brillo, ...",Valentino,Labios,Gloss/ Brillo labial,39.99,1020,4.7,6
89,Easy Bake Setting Spray,Spray fijador de maquillaje 16 horas,Huda Beauty,Rostro,Primer & spray fijador,35.00,1014,4.6,2


In [446]:
outliers_numero_valoraciones.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,256.0,36.228828,12.815420,9.99,27.7425,35.0,42.0,79.0
numero_valoraciones,256.0,3380.621094,3378.514686,1014.00,1397.5000,2292.5,3711.0,20035.0
valoracion,256.0,4.321094,0.307591,2.90,4.2000,4.4,4.6,4.8
numero_variaciones,256.0,4.949219,7.976028,0.00,0.0000,2.0,7.0,49.0


In [447]:
outliers_numero_valoraciones.describe(include='object').T

,count,unique,top,freq
nombre,256,246,Spike Valentino,2
descripcion,251,210,Máscara,6
nombre_marca,256,36,Benefit Cosmetics,45
nombre_categoria,256,8,Rostro,109
nombre_subcategoria,254,32,Máscara de pestañas,43


Los productos con un número de valoraciones excepcionalmente alto (outliers) tienen una mediana de 2,292.5 valoraciones, muy por encima de la mediana general de 42.5, lo que indica una altísima popularidad. No se corresponden con ser productos más baratos (mediana de 35.00, casi igual a los 34.99 generales), y presentan una valoración igual a los productos generales (mediana de 4,4), lo que sugiere una percepción positiva mantenida incluso con un volumen masivo de opiniones. En cuanto a las variaciones, su mediana es de 2.0, igual que la de los productos generales, lo que indica una disponibilidad de opciones promedio. A nivel de marcas, destacan nombres populares como Benefit Cosmetics, y en categorías, predominan productos de Rostro, especialmente la Máscara de pestañas, que aparece como la subcategoría más frecuente. En conjunto, estos productos mantienen un precio promedio en la tienda y son muy bien reconocidos, manteniendo buenas valoraciones a gran escala.

In [247]:
pio.renderers.default = "browser"

fig = px.box(df, y="numero_valoraciones", hover_data=["id_producto"])
fig.update_layout(title="Boxplot del número de valoraciones con ID del producto")
fig.show()

In [448]:
df[df["id_producto"].isin([987, 1574])]

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
988,https://www.sephora.es/p/brow-wiz---l%c3%a1piz...,987,Brow Wiz,Lápiz de cejas,Anastasia Beverly Hills,Cejas,Lápiz & polvo para cejas,36.0,20035,4.5,8
1579,https://www.sephora.es/p/radiant-creamy-concea...,1574,Radiant Creamy Concealer,Corrector de imperfecciones,Nars,Rostro,Antiojeras & corrector,36.0,19235,4.4,14


Al analizar los productos que destacan como outliers por tener un elevado número de valoraciones, se identifican dos que sobresalen ligeramente por encima del resto: un lápiz para cejas de la marca Anastasia Beverly Hills y un antiojeras y corrector de la marca NARS. Ambos no solo cuentan con un número excepcionalmente alto de valoraciones, sino que además mantienen una valoración media muy elevada, lo que indica un éxito significativo en términos tanto de popularidad como de satisfacción del cliente.

Dentro del conjunto más amplio de productos con un número alto de valoraciones —un total de 256 considerados como outliers en esta métrica—, las subcategorías más representadas son la máscara de pestañas (con 43 productos) y la marca Benefit Cosmetics (con 45 productos).

Es importante matizar que el hecho de tener un número elevado de valoraciones no implica necesariamente que el producto sea de alta calidad o esté especialmente bien valorado. Por ello, para evaluar el éxito real de un producto, debe considerarse no solo el volumen de valoraciones recibidas, sino también su puntuación media.

In [451]:
outliers_valoracion.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
8,Maracuja Juicy Lip Duo,Dúo para los labios,Tarte,Estuches maquillaje,Estuches de labiales,29.0,1,3.0,0
175,Dreamglow,Sérum Iluminador,Rem Beauty,Rostro,Iluminador,29.0,2,3.0,3
216,Terracotta Le Teint Glow,Fondo de maquillaje luminosidad natural,Guerlain,Rostro,Base de maquillaje,61.0,2,3.0,30
278,Green Flash,Estuche Let's Flash Poppy Red,Manucurist,Estuches maquillaje,Estuches de manicura,85.0,1,3.0,0
299,Moving Lights,Kit De 3 Minipinceles,Sephora Collection,Pinceles & accesorios,Estuche de pinceles,21.0,6,1.8,0


In [452]:
outliers_valoracion.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,58.0,31.443966,15.384585,6.99,21.25,29.495,39.4925,85.0
numero_valoraciones,58.0,207.482759,905.285149,1.00,1.25,3.000,14.7500,4884.0
valoracion,58.0,2.410345,0.713206,1.00,2.00,2.700,3.0000,3.1
numero_variaciones,58.0,3.517241,6.361180,0.00,0.00,0.000,4.0000,30.0


In [453]:
outliers_valoracion.describe(include='object').T

,count,unique,top,freq
nombre,58,55,Green Flash,2
descripcion,54,54,Dúo para los labios,1
nombre_marca,58,29,Sephora Collection,8
nombre_categoria,57,7,Rostro,18
nombre_subcategoria,56,28,Máscara de pestañas,6


Los productos con valoraciones excepcionalmente bajas presentan una mediana de 0.0, muy por debajo de la mediana general de 4.4, lo que indica una percepción claramente negativa o problemas de calidad o experiencia. Además, estos productos prácticamente no tienen valoraciones (mediana también de 0), lo que refuerza la baja visibilidad o el desinterés por parte de los usuarios. A nivel de precio, su mediana (36.0) es similar a la del conjunto general (34.99), por lo que no son ni especialmente baratos ni caros. En cuanto a variaciones, la mediana es de 0, también por debajo del global (2.0), lo que sugiere menor diversidad de opciones o formatos. Predominan marcas como Sephora Collection, y las categorías más frecuentes son productos de Rostro, destacando subcategorías como Barra de labios o Lápiz de labios. En conjunto, se trata de productos con escasa interacción y muy baja valoración, lo que puede indicar desinterés, problemas de lanzamiento o experiencias de usuario deficientes.

In [454]:
sin_valoraciones = df[df["valoracion"] == 0]
sin_valoraciones.describe(include='object').T

,count,unique,top,freq
url_producto,232,232,https://www.sephora.es/p/cherry-mouth---estuch...,1
nombre,229,218,Moving Lights,4
descripcion,208,186,Esponja de maquillaje,3
nombre_marca,232,51,Dior,23
nombre_categoria,229,10,Rostro,58
nombre_subcategoria,226,40,Barra de labios,23


In [455]:
pio.renderers.default = "browser"

fig = px.box(df_con_valoraciones, y="valoracion", hover_data=["id_producto"])
fig.update_layout(title="Boxplot de las valoraciones con ID del producto")
fig.show()

Al analizar los productos con una valoración igual a 0 —lo que indica que no han recibido ninguna valoración por parte de los usuarios— se identifican un total de 232 productos. Dentro de estos, la subcategoría más frecuente es la de barras de labios, mientras que la marca con mayor representación es Dior. En este caso, no hay ningún producto que destaque por encima del resto, ya que todos comparten el hecho de no haber sido valorados aún.

Por otro lado, si nos centramos en aquellos productos que sí han recibido alguna valoración (es decir, con un mínimo de 1), tampoco se observa un único producto que sobresalga claramente por tener la peor valoración, ya que varios comparten una puntuación de 1. No obstante, dentro de estos productos con valoraciones bajas, se aprecia que la subcategoría más frecuente entre los outliers es la de máscara de pestañas (6 productos), y la marca que más se repite es Sephora Collection (8 productos). Aun así, únicamente hay un producto que coincide tanto en ser una máscara de pestañas como en pertenecer a Sephora Collection.

Esto puede explicarse por el hecho de que Sephora Collection es la marca con mayor número de productos en el catálogo, lo que naturalmente aumenta su probabilidad de aparecer tanto en los productos mejor valorados como en los peor valorados, simplemente por su volumen de presencia.

In [456]:
outliers_numero_variaciones.head()

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
0,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7,50
1,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5,51
2,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1,30
13,Dior Addict,Barra de labios brillante 90 % de ingredientes...,Dior,Labios,Barra de labios,51.00,977,4.5,26
34,Vanish™ Airbrush Concealer,Corrector antiojeras,Hourglass,Rostro,Antiojeras & corrector,42.00,3247,4.3,29


In [457]:
outliers_numero_variaciones.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,114.0,41.629386,16.368728,4.99,32.0,41.49,51.0,99.0
numero_valoraciones,114.0,1048.956140,2804.771208,0.00,20.0,69.50,585.5,17079.0
valoracion,114.0,4.103509,1.064880,0.00,4.1,4.30,4.6,5.0
numero_variaciones,114.0,25.885965,9.463258,16.00,19.0,23.00,30.0,60.0


In [458]:
outliers_numero_variaciones.describe(include='object').T

,count,unique,top,freq
nombre,114,110,Rouge Is Not My Name,2
descripcion,112,81,Corrector,12
nombre_marca,114,34,Sephora Collection,13
nombre_categoria,114,4,Rostro,82
nombre_subcategoria,114,14,Base de maquillaje,51


Los productos con un número inusualmente alto de variaciones (mediana de 23 frente a 2 en el conjunto general) destacan por ofrecer una gran diversidad de tonos, formatos o versiones. A pesar de su complejidad de catálogo, mantienen una valoración sólida (mediana de 4.3, similar al total), lo que indica que la variedad no compromete la calidad percibida. Además, presentan un volumen notable de valoraciones (mediana de 69.5 frente a 42.5 en general), lo que sugiere buena aceptación por parte de los usuarios. Su precio (mediana de 41.49) es algo superior al promedio del catálogo, posiblemente asociado a la amplitud de oferta. Estos productos se concentran sobre todo en la categoría de Rostro, especialmente en la subcategoría de Base de maquillaje, y destacan marcas como Sephora Collection. En conjunto, se trata de productos versátiles y bien valorados, con alta interacción del cliente.

In [459]:
pio.renderers.default = "browser"

fig = px.box(df, y="numero_variaciones", hover_data=["id_producto"])
fig.update_layout(title="Boxplot del número de variaciones con ID del producto")
fig.show()

In [460]:
df[df["id_producto"].isin([764, 903])]

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
764,https://www.sephora.es/p/sweetener---base-de-m...,764,Sweetener,Base de maquillaje con ácido hialurónico,Rem Beauty,Rostro,Base de maquillaje,32.0,15,4.7,60
903,https://www.sephora.es/p/sweetener---corrector...,903,Sweetener,Corrector con ácido hialurónico hidratante,Rem Beauty,Rostro,Antiojeras & corrector,23.0,5,4.8,59


Dentro del análisis de los productos que destacan como outliers por número de variaciones disponibles, hay dos que sobresalen por encima del resto: una base de maquillaje y un antiojeras y corrector, ambos de la marca Rem Beauty. Es interesante destacar que estas dos subcategorías coinciden con las que presentan el mayor número mediano de variaciones en el catálogo de Sephora, lo cual refuerza su presencia en este conjunto de productos. Sin embargo, la marca Rem Beauty no es la que, en términos generales, ofrece más variaciones entre sus productos. Esto puede deberse a que, aunque la marca en sí no tenga un catálogo especialmente amplio, puede concentrar su estrategia de producto en pocas referencias con muchas opciones (por ejemplo, distintos tonos o acabados), lo que eleva su número de variaciones puntuales en algunos artículos.

De forma más general, entre los productos considerados outliers por número de variaciones (en total, 114), la subcategoría más frecuente es la de bases de maquillaje, con 51 productos. Esto concuerda con el hecho de que las bases suelen ofrecer múltiples tonos para adaptarse a distintos tipos de piel. Por otro lado, la marca más presente entre estos outliers es Sephora Collection, con 13 productos.

Aunque Sephora Collection no destaca por tener, en promedio, un número elevado de variaciones por producto ni por centrarse especialmente en las bases de maquillaje como subcategoría principal, su alta presencia en esta lista puede explicarse por el hecho de que es la marca que más productos ofrece en el catálogo de Sephora. Esto aumenta estadísticamente la probabilidad de que aparezca en análisis de outliers en distintas métricas, incluso aunque no sea líder en cada una de ellas de forma proporcional.

### Valores representativos de las variables categóricas

In [259]:
df.describe(include='object').T

,count,unique,top,freq
url_producto,1822,1822,https://www.sephora.es/p-640056.html,1
nombre,1819,1646,Moving Lights,14
descripcion,1724,1204,Paleta de sombras de ojos,44
nombre_marca,1822,79,Sephora Collection,198
nombre_categoria,1813,13,Rostro,627
nombre_subcategoria,1789,48,Máscara de pestañas,148


In [260]:
numero_marcas = df["nombre_marca"].nunique()
print(f"Sephora cuenta con {numero_marcas} marcas diferentes")
numero_categorias = df["nombre_categoria"].nunique()
print(f"Sephora cuenta con {numero_categorias} categorias diferentes")
numero_subcategorias = df["nombre_subcategoria"].nunique()
print(f"Sephora cuenta con {numero_subcategorias} subcategorias diferentes")

Sephora cuenta con 79 marcas diferentes
Sephora cuenta con 13 categorias diferentes
Sephora cuenta con 48 subcategorias diferentes


### Recuento de productos por marca

In [461]:
df_marca_count = df.groupby('nombre_marca').size().reset_index(name='count')
df_marca_count['%/Total'] = ((df_marca_count['count'] / numero_total_productos) * 100).round(2)
df_marca_count = df_marca_count.sort_values(by='%/Total', ascending=False)
df_marca_count.head(5)

,nombre_marca,count,%/Total
63,Sephora Collection,198,10.87
5,Benefit Cosmetics,86,4.72
10,Charlotte Tilbury,77,4.23
42,Make Up For Ever,66,3.62
15,Dior,65,3.57


Sephora Collection es la marca que más presencia tiene en número de productos ofertados en Sephora (cuenta con el 10,87% de los productos), seguida de Benefit Cosmetics y Charlotte Tilbury (4,72% y 4,23% respectivamente). Por el lado contrario, marcas como The Ordinary, Urban Decay, YEPODA y otras más, cuentan con tan solo el 0,05% de productos ofertados en Sephora.

### Recuento de productos por categoria 

In [462]:
df_cat_count = df.groupby('nombre_categoria').size().reset_index(name='count')
df_cat_count['%/Total'] = ((df_cat_count['count'] / numero_total_productos) * 100).round(2)
df_cat_count = df_cat_count.sort_values(by='%/Total', ascending=False)
df_cat_count.head(5)

,nombre_categoria,count,%/Total
10,Rostro,627,34.41
4,Ojos,372,20.42
3,Labios,343,18.83
8,Pinceles & accesorios,167,9.17
0,Cejas,100,5.49


La categoría con más presencia en Sephora es el Rostro con un 34,41% de productos ofertados en la tienda. Le siguen los Ojos con un 20,42% y los Labios con un 18,83%. Por el lado contrario, las categorías con menos presencia serían los Sacapuntas y Pestañas postizas con un 0,22%, los Rizadores de pestañas con un 0,16% y los Papeles matificantes con un 0,05%.

### Recuento de procuctos por subcategoria 

In [463]:
df_subcat_count = df.groupby('nombre_subcategoria').size().reset_index(name='count')
df_subcat_count['%/Total'] = ((df_subcat_count['count'] / numero_total_productos) * 100).round(2)
df_subcat_count = df_subcat_count.sort_values(by='%/Total', ascending=False)
df_subcat_count.head()

,nombre_subcategoria,count,%/Total
32,Máscara de pestañas,148,8.12
2,Barra de labios,126,6.92
8,Colorete,113,6.20
4,Base de maquillaje,107,5.87
42,Primer & spray fijador,104,5.71


Las subcategorías con más presencia dentro de Sephora son las Máscaras de pestañas (8,12%), las Barras de labios (6,92%) y los Coloretes (6,20%). Por el lado contrario, las que tienen menos presencia serían los Limpiadores de pinceles y las Paletas multi-usos con un 0,05% cada una.

Hasta ahora el éxito de los productos se ha estado observando por separado. Por un lado, se ha hecho hincapié en la valoración de los productos y por otro lado en el número de valoraciones por producto. Considero que para poder determinar un producto realmente exitoso se deben observar ambas variables en conjunto ya que, que un producto tenga muchas valoraciones en este caso puede indicar o que el producto gusta mucho (buena valoración) y por eso los consumidores se paran a comentar en el producto, o por el contrario, que el producto gusta poco (mala valoración) y por ello también se paran a comentarlo. Por otro lado, que un producto tenga una valoración muy alta no tiene por qué ser indicador de producto muy exitoso si tan solo cuenta con 2 reseñas por ejemplo, al igual que un producto con una valoración algo más baja no tiene por qué indicar que es peor si sin embargo este cuenta con 100 valoraciones. Por todo ello, para determinar los productos realmente exitosos se va a observar en conjunto el número de valoraciones y la valoración. 

Justificación de los umbrales para considerar productos "exitosos" en Sephora:

Para definir qué productos pueden considerarse exitosos en Sephora, he establecido criterios basados en las variables de valoración media y número de valoraciones, eligiendo umbrales informados por los estadísticos de la distribución general, pero también guiados por criterios prácticos de exploración.

- En cuanto a la valoración, he fijado un umbral en 4,5, que ya puede considerarse una valoración alta (la mediana está en 4,4). Aunque el percentil 75 está en 4,7, una puntuación de 4,5 sitúa al producto cerca del cuartil superior, reflejando una percepción positiva generalizada sin llegar al nivel de productos excepcionalmente valorados (outliers).

- Para el número de valoraciones, he establecido el umbral en 50 valoraciones. La mediana se encuentra en torno a 42, y aunque este umbral es ligeramente superior, permite asegurar que el producto ha sido evaluado por una base mínima de usuarios, sin restringir la muestra solo a aquellos pocos productos con cifras muy elevadas que podrían considerarse atípicos.

Además, dado que el objetivo es explorar y analizar un conjunto representativo y suficientemente amplio de productos exitosos, no conviene ser excesivamente restrictivo. Si se utilizaran directamente los percentiles 75 como umbrales, el resultado sería un grupo demasiado reducido de productos —cercano al comportamiento de outliers— que no reflejaría bien el éxito "habitual" o recurrente dentro del catálogo. Por tanto, este enfoque permite capturar productos que combinan buena valoración y reconocimiento, pero sin limitarse únicamente a casos excepcionales.

### Número y porcentaje de productos exitosos en Sephora

In [464]:
exitosos = df[(df["valoracion"] >= 4.5) & (df["numero_valoraciones"] >= 50)]

print("Número de productos exitosos:", len(exitosos))
print("Porcentaje de productos exitosos:", round((len(exitosos) / numero_total_productos) * 100, 2))

Número de productos exitosos: 393
Porcentaje de productos exitosos: 21.57


### Productos exitosos por marca

In [567]:
total_por_marca = df.groupby('nombre_marca').size().reset_index(name='total_productos')
exitosos_por_marca = exitosos.groupby('nombre_marca').size().reset_index(name='count')
exitosos_por_marca['%/Total_exitosos'] = ((exitosos_por_marca['count'] / len(exitosos)) * 100).round(2)
exitosos_por_marca = exitosos_por_marca.merge(total_por_marca, on='nombre_marca', how='left')
exitosos_por_marca['%_exitosos_sobre_marca'] = ((exitosos_por_marca['count'] / exitosos_por_marca['total_productos']) * 100).round(2)
exitosos_por_marca = exitosos_por_marca.sort_values(by='%/Total_exitosos', ascending=False)
exitosos_por_marca.head(5)

,nombre_marca,count,%/Total_exitosos,total_productos,%_exitosos_sobre_marca
3,Benefit Cosmetics,35,8.91,86,40.70
37,Sephora Collection,32,8.14,198,16.16
30,Natasha Denona,29,7.38,56,51.79
0,Anastasia Beverly Hills,27,6.87,61,44.26
19,Huda Beauty,23,5.85,47,48.94


- Marcas con más productos exitosos en términos absolutos
Benefit Cosmetics es la marca con mayor número de productos exitosos (35), seguida por Sephora Collection (32), Natasha Denona (29), Anastasia Beverly Hills (27) y Huda Beauty (23).
Estas marcas concentran una parte significativa del total de productos exitosos, representando entre un 6 y un 9% cada una sobre el total de éxitos de Sephora.
- Porcentaje de productos exitosos respecto al catálogo de cada marca: algunas marcas tienen un porcentaje muy alto de éxito dentro de su propio catálogo.
    - Glow Recipe y Pai → 100% de sus productos son exitosos (aunque tienen catálogos muy pequeños: 2 y 3 productos).
    - Caudalie → 66.67% (2 de 3).
    - Natasha Denona → 51.79% (29 de 56).
    - Huda Beauty → 48.94% (23 de 47).
    - Anastasia Beverly Hills → 44.26%.
En cambio, Sephora Collection, aunque tiene muchos productos exitosos en términos absolutos, solo un 16.16% de su catálogo lo es, debido a su gran volumen total (198 productos).

### Productos exitosos por categorías

In [566]:
total_por_cat = df.groupby('nombre_categoria').size().reset_index(name='total_productos')
exitosos_por_cat = exitosos.groupby('nombre_categoria').size().reset_index(name='count')
exitosos_por_cat['%/Total_exitosos'] = ((exitosos_por_cat['count'] / len(exitosos)) * 100).round(2)
exitosos_por_cat = exitosos_por_cat.merge(total_por_cat, on='nombre_categoria', how='left')
exitosos_por_cat['%_exitosos_sobre_categoria'] = ((exitosos_por_cat['count'] / exitosos_por_cat['total_productos']) * 100).round(2)
exitosos_por_cat = exitosos_por_cat.sort_values(by='%_exitosos_sobre_categoria', ascending=False)
exitosos_por_cat.head(5)

,nombre_categoria,count,%/Total_exitosos,total_productos,%_exitosos_sobre_categoria
5,Paletas maquillaje,20,5.09,49,40.82
6,Pinceles & accesorios,46,11.70,167,27.54
0,Cejas,26,6.62,100,26.00
8,Sacapuntas,1,0.25,4,25.00
7,Rostro,152,38.68,627,24.24


- Categorías con mayor volumen de productos exitosos (peso entre los éxitos totales): Rostro es, con diferencia, la categoría más representada entre los productos exitosos. 152 productos (38.68% del total de productos exitosos). Tiene también un alto catálogo total (627), lo que indica que es una categoría central en Sephora. Le siguen:
    - Labios (69 productos, 17.56% del total de éxitos).
    - Ojos (63 productos, 16.03%).
    - Pinceles & accesorios (46 productos, 11.70%).
    - Cejas (26 productos, 6.62%).

- Categorías más eficientes internamente (mayor % de éxito respecto a su propio catálogo). La categoría más eficiente es: Paletas de maquillaje → 40.82% de sus productos son exitosos, aunque solo representan el 5.09% del total de éxitos. Le siguen:
    - Pinceles & accesorios → 27.54%.
    - Cejas → 26.00%.
    - Sacapuntas (con 1 de 4 productos exitosos) → 25.00%.
    - Rostro → 24.24%.

### Productos exitosos por subcategorías

In [565]:
total_por_subcat = df.groupby('nombre_subcategoria').size().reset_index(name='total_productos')
exitosos_por_subcat = exitosos.groupby('nombre_subcategoria').size().reset_index(name='count')
exitosos_por_subcat['%/Total_exitosos'] = ((exitosos_por_subcat['count'] / len(exitosos)) * 100).round(2)
exitosos_por_subcat = exitosos_por_subcat.merge(total_por_subcat, on='nombre_subcategoria', how='left')
exitosos_por_subcat['%_exitosos_sobre_subcat'] = ((exitosos_por_subcat['count'] / exitosos_por_subcat['total_productos']) * 100).round(2)
exitosos_por_subcat = exitosos_por_subcat.sort_values(by='%_exitosos_sobre_subcat', ascending=False)
exitosos_por_subcat.head()

,nombre_subcategoria,count,%/Total_exitosos,total_productos,%_exitosos_sobre_subcat
11,Cremas BB & CC,9,2.29,18,50.00
26,Paleta rostro,7,1.78,15,46.67
25,Paleta de sombras de ojos,29,7.38,74,39.19
8,Colorete,43,10.94,113,38.05
14,Estuche de pinceles,4,1.02,11,36.36


- Subcategorías con mayor volumen de productos exitosos (peso en los éxitos totales): Colorete lidera el ranking con 43 productos exitosos, representando un 10.94% del total de productos exitosos en Sephora. Tiene un catálogo considerable de 113 productos, lo que indica que es una subcategoría importante. Le siguen:
    - Brocha rostro con 30 productos exitosos (7.63% del total).
    - Paleta de sombras de ojos con 29 productos (7.38%).
    - Barra de labios con 28 productos (7.12%).
    - Sombras de ojos con 23 productos (5.85%).
- Subcategorías más eficientes internamente (mayor % de éxito respecto a su propio catálogo). Las subcategorías con mayor proporción de productos exitosos sobre su total son:
    - Cremas BB & CC con un 50.00% de éxito (9 de 18 productos).
    - Paleta rostro con 46.67% de éxito (7 de 15 productos).
    - Paleta de sombras de ojos con 39.19% (29 de 74 productos).
    - Colorete con 38.05%.
    - Estuche de pinceles con 36.36% (4 de 11 productos).

In [469]:
mediana_precio_exitosos = exitosos["precio"].median()
mediana_precio_exitosos

np.float64(36.0)

Al ser la mediana del precio de los productos exitosos (36 euros) prácticamente el mismo que el de los productos de Sephora en general (37,20 euros), sugiere lo que ya se había comenzado a observar en los outliers, que precios más bajos no son la justificación del éxito en los productos. Aquellos productos con precios dentro del rango habitual de Sephora son los más exitosos.

In [470]:
mediana_variaciones_exitosos = exitosos["numero_variaciones"].median()
mediana_variaciones_exitosos

np.float64(2.0)

### Precio mediano por marcas

In [471]:
precio_mediano_marca = (df.groupby("nombre_marca")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_marca.head()

,nombre_marca,precio
38,La Mer,102.50
66,Sisley,65.50
64,Sephora Favorites,65.00
8,CHANEL,55.50
30,Helena Rubinstein,54.99


La marca con el precio mediano más alto es La Mer (102,50 euros), seguida de Sisley (65,50 euros) y Sephora Favorites (65,00). Por el lado contrario, las marcas con el precio medianos más bajo son Youth To The People (15,00), SVR (15,00), The Ordinary (14,99), Caudalie (14,00), Sephora Collection (13,99), Merci Handy (12,99).

In [472]:
precio_mediano_marca[precio_mediano_marca["nombre_marca"] == "Benefit Cosmetics"]

,nombre_marca,precio
5,Benefit Cosmetics,33.0


In [473]:
precio_mediano_marca[precio_mediano_marca["nombre_marca"] == "Erborian"]

,nombre_marca,precio
19,Erborian,30.0


In [474]:
precio_mediano_marca[precio_mediano_marca["nombre_marca"] == "Natasha Denona"]

,nombre_marca,precio
49,Natasha Denona,30.49


De nuevo, que Benefit Cosmetics tenga un precio medio superior a Sephora Collection y aún así cuente con un mayor número de productos exitosos indica lo ya observado -> Un precio más bajo no tiene por qué ser justificante de los productos con mayor éxito.

### Precio mediano por categorías

In [475]:
precio_mediano_categoria = (df.groupby("nombre_categoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_categoria.tail()

,nombre_categoria,precio
7,Pestañas postizas,22.00
6,Papeles matificantes,20.00
9,Rizador pestañas,15.99
12,Uñas,15.99
11,Sacapuntas,10.00


Las categorías con precios medianos más altos son las Paletas de maquillaje (59,00 euros), el Rostro (40,99) y los Ojos (33,00), mientras que las categorías con precios medianos más bajos son los Rizadores de pestañas (15,99), las Uñas (15,99), y los Sacapuntas (10,00).

### Precio mediano por subcategorías

In [476]:
precio_mediano_subcategoria = (df.groupby("nombre_subcategoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_subcategoria.head()

,nombre_subcategoria,precio
36,Paleta rostro,64.99
34,Paleta de sombras de ojos,59.00
33,Paleta de contouring,55.00
4,Base de maquillaje,51.00
35,Paleta multi-usos,51.00


Las subcategorías con precios medianos más altos son las Paletas de rostro (64,99), las Paletas de sombras de ojos (59,00) y las Paletas de contouring (55,00). Por el lado contrario, las subcategorías con precios medianos más bajos son los Esmaltes de uñas (13,99), los Quitaesmaltes (13,99) y los Limpiadores de pinceles (8,99).

In [477]:
precio_mediano_subcategoria[precio_mediano_subcategoria["nombre_subcategoria"] == "Cremas BB & CC"]

,nombre_subcategoria,precio
11,Cremas BB & CC,39.0


### Marcas que dominan cada categoría de productos

In [478]:
marcas_por_categoria = df.groupby(["nombre_categoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")
marcas_dominantes_por_categoria = marcas_por_categoria.loc[marcas_por_categoria.groupby("nombre_categoria")["numero_productos"].idxmax()]
marcas_dominantes_por_categoria.head()

,nombre_categoria,nombre_marca,numero_productos
0,Cejas,Anastasia Beverly Hills,21
35,Desmaquillante,Collistar,5
77,Estuches maquillaje,Sephora Collection,9
127,Labios,Sephora Collection,24
168,Ojos,Sephora Collection,53


### Marcas que dominan cada subcategoría de productos

In [554]:
marcas_por_subcategoria = df.groupby(["nombre_subcategoria", "nombre_marca"])["url_producto"].count().reset_index(name="numero_productos")
marcas_dominantes_por_subcategoria = marcas_por_subcategoria.loc[marcas_por_subcategoria.groupby("nombre_subcategoria")["numero_productos"].idxmax()]
marcas_dominantes_por_subcategoria.head()

,nombre_subcategoria,nombre_marca,numero_productos
0,Aceite desmaquillante,CHANEL,2
47,Antiojeras & corrector,Tarte,9
54,Barra de labios,Charlotte Tilbury,11
83,Barra de labios líquida,Benefit Cosmetics,3
107,Base de maquillaje,Fenty Beauty,7


### Número de variaciones mediano por marca

In [480]:
variaciones_median_marca = (df.groupby("nombre_marca")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_marca.head()

,nombre_marca,numero_variaciones
52,Olehenriksen,8.0
24,Gisou,7.0
1,Armani,6.0
36,KOSAS,6.0
40,Laneige,6.0


### Número de variaciones mediano por categoría

In [481]:
variaciones_median_cat = (df.groupby("nombre_categoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_cat.head()

,nombre_categoria,numero_variaciones
3,Labios,5.0
10,Rostro,4.0
0,Cejas,3.0
2,Estuches maquillaje,0.0
1,Desmaquillante,0.0


### Número de variaciones mediano por subcategoría

In [482]:
variaciones_median_subcat = (df.groupby("nombre_subcategoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_subcat.head()

,nombre_subcategoria,numero_variaciones
4,Base de maquillaje,15.0
1,Antiojeras & corrector,10.5
10,Crema de color,9.5
24,Gel uñas,9.5
30,Lápiz de labios,8.0


Ahora, con un análisis un poco más específico de Sephora Collection dentro de Sephora, va a permitir terminar de posicionar a Sephora Collection dentro de la tienda para tener esa primera visión clara de la marca y poder pasar posteriormente ya a decidir sobre el producto a lanzar en su próximo lanzamiento.

## Análisis Específico Sephora Collection

In [483]:
df_especifico = df[df["nombre_marca"] == "Sephora Collection"]
df_especifico.head()

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
11,https://www.sephora.es/p/el-estuche-de-brochas...,12,El Estuche de Brochas de Iniciación,"Aplicación intuitiva, acabado perfecto",Sephora Collection,Pinceles & accesorios,Estuche de pinceles,42.99,94,4.9,0
48,https://www.sephora.es/p/get-unready*---estuch...,49,Get (Un)Ready*,Estuche trío belleza,Sephora Collection,Estuches maquillaje,Estuches multi-usos,24.00,0,0.0,0
51,https://www.sephora.es/p/time-to-glitter---pur...,52,Time To Glitter,Purpurina Instantánea 2 geles de purpurina,Sephora Collection,None,None,18.00,0,0.0,0
59,https://www.sephora.es/p/size-up---m%c3%a1scar...,60,Size Up,Máscara Volumen Extralargo Inmediato (Formato ...,Sephora Collection,Ojos,Máscara de pestañas,7.00,640,3.9,0
60,https://www.sephora.es/p/colorful-luminizer---...,61,Colorful Luminizer,Polvos iluminadores para el rostro,Sephora Collection,Rostro,Iluminador,12.99,0,0.0,3


In [484]:
df_especifico.shape

(198, 11)

Como ya se sabía del análisis general, Sephora Collection cuenta con 198 productos en su catálogo.

In [485]:
info_reporte(df_especifico)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
url_producto,0,0.00,object
id_producto,0,0.00,int64
nombre,0,0.00,object
descripcion,21,10.61,object
nombre_marca,0,0.00,object
nombre_categoria,2,1.01,object
nombre_subcategoria,5,2.53,object
precio,0,0.00,float64
numero_valoraciones,0,0.00,int64
valoracion,0,0.00,float64


De estos 198 productos, 21 carecen de descripción, 2 de categoría y 5 de subcategoría.

In [486]:
df_especifico["url_producto"].duplicated().sum()

np.int64(0)

De los 7 duplicados que se observaron, ninguno de ellos pertenece a la marca de Sephora Collection por lo que en este caso no hay que eliminar ningún duplicado.

In [487]:
df_especifico.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,198.0,650.424242,468.213281,12.00,301.25,502.00,849.75,1761.00
precio,198.0,15.846061,7.564304,3.99,11.99,13.99,16.99,59.99
numero_valoraciones,198.0,166.823232,542.807371,0.00,6.25,26.00,97.75,5428.00
valoracion,198.0,3.914141,1.406259,0.00,4.00,4.40,4.70,5.00
numero_variaciones,198.0,3.484848,7.258622,0.00,0.00,0.00,4.00,48.00


Descriptivos de las variables numéricas: 
- Precio: Los productos de Sephora Collection tienen un precio claramente más accesible dentro del catálogo de Sephora. Oscilan entre 3,99 € y 59,99 €, con una media de 15,85 € y una mediana aún más baja, de 13,99 €. Esto refleja una distribución ligeramente sesgada a la derecha, donde la mayoría de los productos tienen precios bajos, pero algunos pocos alcanzan precios más elevados, aunque muy por debajo del rango general de la tienda. El 25 % de los productos cuesta menos de 11,99 €, el 50 % menos de 13,99 €, y el 75 % menos de 16,99 €, lo que confirma la orientación de esta marca hacia productos de gama económica.
- Número de valoraciones: La cantidad de valoraciones varía significativamente, desde 0 hasta 5.428. Sin embargo, la media (166,82) está muy por encima de la mediana (26), lo que indica una distribución altamente sesgada a la derecha. Esto significa que mientras algunos productos son muy populares, con miles de valoraciones, la mayoría recibe muchas menos. Un 25 % tiene menos de 6,25 valoraciones y solo una cuarta parte supera las 97,75, lo que refleja una popularidad desigual entre productos, aunque existen casos destacados de gran visibilidad.
- Valoración: Las puntuaciones de los productos oscilan entre 0 y 5. La media es de 3,91, mientras que la mediana es de 4,40, lo que indica una ligera asimetría hacia la izquierda. Esto implica que la mayoría de los productos tienen buenas valoraciones, aunque algunos con puntuaciones muy bajas tiran la media hacia abajo. El 25 % de los productos tiene una valoración inferior a 4, el 50 % no supera los 4,4, y el 75 % se sitúa por debajo de 4,7. Esto sugiere que, en general, los productos de Sephora Collection son bien recibidos por los clientes.
- Número de variaciones: En cuanto a la variedad de opciones (como tonos o tamaños), los productos de Sephora Collection muestran una distribución muy desigual. Aunque el promedio de variaciones por producto es de 3,48, la mediana es 0, lo que indica que más de la mitad de los productos no presenta ninguna variación. El 25 % tiene 0 variaciones, el 75 % no supera las 4, y unos pocos productos alcanzan hasta 48 variaciones. Esto refleja que aunque la mayoría de los productos son simples en su oferta, existen algunos casos específicos con una gran diversidad.

### Valores representativos de las variables numéricas

In [511]:
mediana_precio_espec = df_especifico["precio"].median()
print(f"Mediana precio: {mediana_precio_espec}")
mediana_num_valoraciones_espec = df_especifico["numero_valoraciones"].median()
print(f"Mediana numero valoraciones: {mediana_num_valoraciones_espec}")
mediana_valoracion_espec = df_especifico["valoracion"].median()
print(f"Mediana valoracion: {mediana_valoracion_espec}")
mediana_num_variaciones_espec = df_especifico["numero_variaciones"].median()
print(f"Mediana numero variaciones: {mediana_num_variaciones_espec}")

Mediana precio: 13.99
Mediana numero valoraciones: 26.0
Mediana valoracion: 4.4
Mediana numero variaciones: 0.0


Evidentemente hay que tener en cuenta que por ejemplo en la mediana del número de variaciones que sale de 0 va a influenciar el hecho de que Sephora Collection ofrezca muchos productos dle tipo brochas de rostro (que por su naturaleza no suelen tener variaciones). Aunque si que pueda tener productos en su catálogo con muchas opciones, el hecho de que una parte grande del catálogo esté compuesta por productos sin variaciones pues tiende a que la mediana sea de cero. 

### Observación de outliers

In [489]:
def detectar_outliers(df, columna, modo='superior'):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    umbral_inferior = Q1 - 1.5 * IQR
    umbral_superior = Q3 + 1.5 * IQR

    if modo == 'superior':
        outliers = df[df[columna] > umbral_superior]
    else:
        outliers = df[df[columna] < umbral_inferior]

    return outliers[['nombre', 'descripcion', 'nombre_marca', 'nombre_categoria', 'nombre_subcategoria', 'precio', 'numero_valoraciones', 'valoracion', 'numero_variaciones']]

df_con_valoraciones_especifico = df_especifico[df_especifico["valoracion"] != 0]

outliers_precio_especifico = detectar_outliers(df_especifico, 'precio', modo='superior')
outliers_numero_valoraciones_especifico = detectar_outliers(df_especifico, 'numero_valoraciones', modo='superior')
outliers_valoracion_especifico = detectar_outliers(df_con_valoraciones_especifico, 'valoracion', modo='inferior')
outliers_numero_variaciones_especifico = detectar_outliers(df_especifico, 'numero_variaciones', modo='superior')

In [289]:
pio.renderers.default = "browser"

fig = px.box(df_especifico, y="precio", hover_data=["id_producto"])
fig.update_layout(title="Boxplot del precio con ID del producto")
fig.show()

In [496]:
df_especifico[df_especifico["id_producto"] == 309]

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
308,https://www.sephora.es/p/el-estuche-completo-d...,309,El Estuche Completo de Brochas,"Aplicación intuitiva, acabado perfecto",Sephora Collection,Pinceles & accesorios,Estuche de pinceles,59.99,83,4.9,0


In [497]:
outliers_precio_especifico.describe(include='object').T

,count,unique,top,freq
nombre,26,17,Moving Lights,8
descripcion,21,18,"Aplicación intuitiva, acabado perfecto",4
nombre_marca,26,1,Sephora Collection,26
nombre_categoria,26,5,Pinceles & accesorios,15
nombre_subcategoria,26,10,Estuche de pinceles,7


Dentro de los productos que destacan como outliers por precio, sí hay uno que sobresale especialmente frente al resto: un estuche de pinceles. Este producto presenta un precio notablemente más elevado en comparación con otros artículos de la misma marca. Curiosamente, esta subcategoría —accesorios como pinceles— no se encuentra entre las cinco subcategorías con el precio mediano más alto dentro del catálogo de Sephora Collection.

Esto puede deberse a que los estuches de pinceles pueden variar considerablemente en precio en función del número de piezas incluidas, la calidad de los materiales, o incluso si se trata de una edición especial o limitada. Así, aunque la subcategoría en su conjunto no tenga un precio mediano elevado, puede haber casos concretos que disparen el precio por encima del resto.

Además, a nivel general —considerando todos los productos que son outliers por precio independientemente de la marca—, la subcategoría de estuches de pinceles es la que más se repite dentro de este grupo, con un total de 7 productos de los 26 identificados como outliers. Esto refuerza la idea de que, aunque no sea la subcategoría más cara en términos generales, sí presenta una alta dispersión de precios, con algunos artículos particularmente costosos que se alejan del resto del conjunto.

In [292]:
pio.renderers.default = "browser"

fig = px.box(df_especifico, y="numero_valoraciones", hover_data=["id_producto"])
fig.update_layout(title="Boxplot del número de valoraciones con ID del producto")
fig.show()

In [498]:
df_especifico[df_especifico["id_producto"].isin([866, 1447])]

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
866,https://www.sephora.es/p/l%c3%a1piz-de-ojos-wa...,866,Lápiz de Ojos Waterproof 12H,None,Sephora Collection,Ojos,Eyeliner,12.99,5428,4.3,15
1452,https://www.sephora.es/p/l%c3%a1piz-de-cejas--...,1447,Lápiz de cejas retráctil,Retractable brow pencil,Sephora Collection,Cejas,Lápiz & polvo para cejas,12.99,4143,4.3,6


In [499]:
outliers_numero_valoraciones_especifico.describe(include='object').T

,count,unique,top,freq
nombre,25,22,Size Up,2
descripcion,16,16,Máscara Volumen Extralargo Inmediato (Formato ...,1
nombre_marca,25,1,Sephora Collection,25
nombre_categoria,25,5,Ojos,11
nombre_subcategoria,25,12,Máscara de pestañas,7


In [500]:
outliers_numero_valoraciones_especifico.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,25.0,15.9108,8.019880,4.99,9.99,13.99,19.99,34.99
numero_valoraciones,25.0,1047.2800,1215.362207,246.00,362.00,640.00,1085.00,5428.00
valoracion,25.0,4.2640,0.313422,3.60,4.10,4.20,4.40,4.80
numero_variaciones,25.0,5.7200,12.367700,0.00,0.00,0.00,6.00,48.00


Dentro de los productos de Sephora Collection que destacan como outliers por número de valoraciones, hay dos que llaman especialmente la atención por estar notablemente alejados del resto: un eyeliner y un lápiz para cejas. Ambos productos cuentan con una valoración de 4,3, lo que indica no solo que han recibido muchas valoraciones —y por tanto han sido muy adquiridos—, sino que también han sido bien valorados por los usuarios. Además, ambos tienen un precio en torno a los 13 euros, lo cual es bastante representativo del rango de precios habitual dentro del catálogo de Sephora Collection. Estos datos sugieren que son productos exitosos tanto en popularidad como en satisfacción del cliente.

Por otro lado, si observamos los productos outliers en general por número de valoraciones (independientemente de la marca), se identifican 25 productos, de los cuales 7 pertenecen a la subcategoría de máscaras de pestañas. Es importante matizar que un alto número de valoraciones no implica necesariamente que el producto sea bueno; sin embargo, en este caso concreto, los valores de valoración de estos outliers oscilan entre 3,6 y 4,8, con una mediana de 4,2. Esto sugiere que, efectivamente, estos productos no solo han sido muy adquiridos, sino que han sido bien recibidos por los usuarios. En consecuencia, en este caso el elevado número de valoraciones sí parece ser un buen indicador de éxito.

In [501]:
df_especifico[df_especifico["valoracion"] == 0].describe(include='object').T

,count,unique,top,freq
url_producto,18,18,https://www.sephora.es/p/get-unready*---estuch...,1
nombre,18,15,Moving Lights,4
descripcion,17,17,Estuche trío belleza,1
nombre_marca,18,1,Sephora Collection,18
nombre_categoria,16,6,Estuches maquillaje,4
nombre_subcategoria,16,13,Estuches multi-usos,3


In [502]:
pio.renderers.default = "browser"

fig = px.box(df_con_valoraciones_especifico, y="valoracion", hover_data=["id_producto"])
fig.update_layout(title="Boxplot de las valoraciones con ID del producto")
fig.show()

In [503]:
outliers_valoracion_especifico

,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
299,Moving Lights,Kit De 3 Minipinceles,Sephora Collection,Pinceles & accesorios,Estuche de pinceles,21.00,6,1.8,0
302,Moving Lights,Kit Duo Labios,Sephora Collection,Labios,Voluminizador labios,13.00,3,1.3,0
358,Pegamento Para Uñas Postizas,None,Sephora Collection,Uñas,Esmalte de uñas,9.99,1,1.0,0
506,Sombra de ojos Colorful,Efecto nacarado,Sephora Collection,Ojos,Sombras de ojos,7.99,2,1.5,4
508,Base de sombra de ojos,Booste + fixe,Sephora Collection,Ojos,Base de sombra de ojos,14.99,27,2.8,0
603,Cuticle Care,Tratamiento para cutículas,Sephora Collection,Uñas,Tratamiento para uñas,10.99,1,1.0,0
1191,Big By Definition Mascara,Máscara Volumen Formato Viaje,Sephora Collection,Ojos,Máscara de pestañas,6.99,15,2.9,0
1577,Pincel para cejas 12,Pincel para cejas,Sephora Collection,Pinceles & accesorios,None,9.99,3,2.3,0


In [504]:
outliers_valoracion_especifico.describe().T

,count,mean,std,min,25%,50%,75%,max
precio,8.0,11.8675,4.489355,6.99,9.490,10.49,13.4975,21.0
numero_valoraciones,8.0,7.2500,9.207916,1.00,1.750,3.00,8.2500,27.0
valoracion,8.0,1.8250,0.762983,1.00,1.225,1.65,2.4250,2.9
numero_variaciones,8.0,0.5000,1.414214,0.00,0.000,0.00,0.0000,4.0


En lo que respecta a las valoraciones de los productos de Sephora Collection, únicamente 18 productos presentan una valoración igual a 0, lo que indica que no han sido valorados por ningún usuario. De estos, 3 son estuches multiusos, un tipo de producto que probablemente tiene menor protagonismo dentro del catálogo y, por tanto, un impacto más limitado en términos de visibilidad o interés por parte de los usuarios.

Por otro lado, si nos centramos en los productos que sí han recibido al menos una valoración (es decir, aquellos con una valoración mínima de 1), no se observa ningún producto que destaque negativamente de forma clara respecto al resto. Tan solo hay 8 productos con valoraciones tan bajas, lo cual es un número reducido teniendo en cuenta el amplio volumen de productos que Sephora Collection ofrece en su catálogo. Además, no se identifica ninguna subcategoría concreta que concentre estos productos mal valorados, sino que están repartidos entre distintas tipologías, lo que indica que no hay un patrón claro de rechazo hacia un tipo de producto en particular.

En términos de popularidad, estos productos tampoco parecen haber recibido un gran número de valoraciones, lo que sugiere que no son artículos especialmente visibles o relevantes dentro de la tienda. Además, presentan precios bajos —con una mediana de 10,49 €—, por lo que el bajo nivel de valoración no parece estar relacionado con el precio.

En definitiva, estos casos parecen formar parte del comportamiento natural dentro de una marca que ofrece una amplia variedad de productos, como es Sephora Collection, donde es esperable que algunos artículos tengan menos éxito o visibilidad sin que ello implique necesariamente un problema de calidad generalizado.

In [300]:
pio.renderers.default = "browser"

fig = px.box(df_especifico, y="numero_variaciones", hover_data=["id_producto"])
fig.update_layout(title="Boxplot del número de variaciones con ID del producto")
fig.show()

In [505]:
df_especifico[df_especifico["id_producto"] == 1385]

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones
1388,https://www.sephora.es/p/best-skin-ever-fond-d...,1385,Base de maquillaje Best Skin Ever,Base de maquillaje Larga Duración,Sephora Collection,Rostro,Base de maquillaje,17.99,1085,4.2,48


In [506]:
outliers_numero_variaciones_especifico.describe(include='object').T

,count,unique,top,freq
nombre,21,20,Rouge Is Not My Name,2
descripcion,20,20,Lápiz de labios sin transferencia,1
nombre_marca,21,1,Sephora Collection,21
nombre_categoria,21,4,Labios,8
nombre_subcategoria,21,9,Base de maquillaje,6


En cuanto al número de variaciones dentro de los productos de Sephora Collection, destaca ligeramente por encima del resto una base de maquillaje, lo cual es coherente con la tendencia general observada en la tienda, donde esta subcategoría tiende a ofrecer una mayor variedad de tonos o formatos.

Además, entre el resto de productos considerados outliers en número de variaciones (un total de 21), las bases de maquillaje vuelven a ser la subcategoría más representada, con 7 productos. Esto refuerza la idea de que se trata de un tipo de producto que, por su naturaleza, requiere una oferta más amplia para adaptarse a las distintas necesidades del cliente (tonos de piel, tipos de acabado, cobertura, etc.).

Este comportamiento también está alineado con la estrategia de muchas marcas —incluida Sephora Collection— de ampliar cada vez más su gama en productos de base para ser más inclusivas y competitivas en el mercado. Por tanto, no solo es esperable, sino que también tiene sentido desde una perspectiva tanto comercial como de experiencia de usuario.

### Valores representativos de las variables categóricas

In [304]:
df_especifico.describe(include="object").T

,count,unique,top,freq
url_producto,198,198,https://www.sephora.es/p/el-estuche-de-brochas...,1
nombre,198,169,Moving Lights,14
descripcion,177,153,"Aplicación intuitiva, acabado perfecto",16
nombre_marca,198,1,Sephora Collection,198
nombre_categoria,196,11,Ojos,53
nombre_subcategoria,193,40,Brocha rostro,25


In [508]:
numero_categorias_espec = df_especifico["nombre_categoria"].nunique()
print(f"Sephora Collection cuenta con {numero_categorias_espec} categorias diferentes")
numero_subcategorias_espec = df_especifico["nombre_subcategoria"].nunique()
print(f"Sephora Collection cuenta con {numero_subcategorias_espec} subcategorias diferentes")

Sephora Collection cuenta con 11 categorias diferentes
Sephora Collection cuenta con 40 subcategorias diferentes


In [509]:
categorias_totales = set(df["nombre_categoria"].dropna().unique())
categorias_sephora_collection = set(df_especifico["nombre_categoria"].dropna().unique())

categorias_no_presentes = categorias_totales - categorias_sephora_collection
print("Categorías no presentes en Sephora Collection:", categorias_no_presentes)

Categorías no presentes en Sephora Collection: {'Pestañas postizas', 'Papeles matificantes'}


In [510]:
subcategorias_totales = set(df["nombre_subcategoria"].dropna().unique())
subcategorias_sephora_collection = set(df_especifico["nombre_subcategoria"].dropna().unique())

subcategorias_no_presentes = subcategorias_totales - subcategorias_sephora_collection
print("Subcategorías no presentes en Sephora Collection:", subcategorias_no_presentes)

Subcategorías no presentes en Sephora Collection: {'Paleta multi-usos', 'Estuche para cejas', 'Estuches de rostro', 'Crema de color', 'Paleta de contouring', 'Gel uñas', 'Cremas BB & CC', 'Estuches de ojos'}


### Recuento productos por categoría

In [512]:
df_cat_count = df_especifico.groupby('nombre_categoria').size().reset_index(name='count')
df_cat_count['%/Total'] = ((df_cat_count['count'] / df_especifico.shape[0]) * 100).round(2)
df_cat_count = df_cat_count.sort_values(by='%/Total', ascending=False)
df_cat_count.head()

,nombre_categoria,count,%/Total
4,Ojos,53,26.77
6,Pinceles & accesorios,48,24.24
8,Rostro,37,18.69
3,Labios,24,12.12
10,Uñas,12,6.06


Las categorías donde Sephora Collection tiene mayor presencia dentro de su catálogo de productos ofertado son en productos para Ojos (26,77% de su oferta), Pinceles & accesorios (24,24%), Rostro (18,69%) y Labios (12,12%). Por el otro extremo, en las categorías donde menor presencia tiene dentro de su oferta son en las siguientes: Desmaquillante (1,52%), Paletas maquillaje (1,01%), Rizador pestañas (0,51%) y Sacapuntas (0,51%).

### Recuento productos por subcategoría

In [513]:
df_subcat_count = df_especifico.groupby('nombre_subcategoria').size().reset_index(name='count')
df_subcat_count['%/Total'] = ((df_subcat_count['count'] / df_especifico.shape[0]) * 100).round(2)
df_subcat_count = df_subcat_count.sort_values(by='%/Total', ascending=False)
df_subcat_count.head()

,nombre_subcategoria,count,%/Total
6,Brocha rostro,25,12.63
36,Sombras de ojos,17,8.59
26,Máscara de pestañas,15,7.58
13,Estuche de pinceles,11,5.56
17,Eyeliner,10,5.05


Las subcategorías donde Sephora Collection tiene mayor presencia dentro de su catálogo de productos ofertado son en las Brochas de rostro (12,63%), las Sombras de ojos (8,59%) y las Máscaras de pestañas (7,58%). Por el otro extremo, en las subcategorías donde menor presencia tiene dentro de su oferta (vendiendo únicamente 1 producto de cada una) son en las siguientes: Base de sombra de ojos, Aceite demaquillante, Barra de labios líquida, Limpiador pinceles, Leche desmaquillante, Desmaquillante rostro, Voluminizador labios. 

### Número y porcentaje de productos exitosos en Sephora Collection

In [514]:
exitosos_espec = df_especifico[(df_especifico["valoracion"] >= 4.5) & (df_especifico["numero_valoraciones"] >= 50)]

print("Número de productos exitosos:", len(exitosos_espec))
print("Porcentaje de productos exitosos:", round((len(exitosos_espec) / df_especifico.shape[0]) * 100, 2))

Número de productos exitosos: 32
Porcentaje de productos exitosos: 16.16


Dentro de Sephora Collection los límites establecidos para determinar productos exitosos dentro de Sephora suponen límites que determinan productos algo más exitosos aún dentro de Sephora Collection ya que la valoración se corresponde, pero el número de valoraciones (50) es más alto que su mediana (26) y por tanto es un límite más exigente.

### Productos exitosos por categoría

In [558]:
total_sc_por_cat = df_especifico.groupby('nombre_categoria').size().reset_index(name='total_productos_SC')
exitosos_sc_por_cat = exitosos_espec.groupby('nombre_categoria').size().reset_index(name='count')
exitosos_sc_por_cat['%/Total_exitosos_SC'] = ((exitosos_sc_por_cat['count'] / len(exitosos_espec)) * 100).round(2)
exitosos_sc_por_cat = exitosos_sc_por_cat.merge(total_sc_por_cat, on='nombre_categoria', how='left')
exitosos_sc_por_cat['%_exitosos_sobre_cat_SC'] = ((exitosos_sc_por_cat['count'] / exitosos_sc_por_cat['total_productos_SC']) * 100).round(2)
exitosos_sc_por_cat = exitosos_sc_por_cat.sort_values(by='%/Total_exitosos_SC', ascending=False)
exitosos_sc_por_cat

,nombre_categoria,count,%/Total_exitosos_SC,total_productos_SC,%_exitosos_sobre_cat_SC
3,Pinceles & accesorios,19,59.38,48,39.58
4,Rostro,6,18.75,37,16.22
2,Ojos,4,12.50,53,7.55
0,Desmaquillante,2,6.25,3,66.67
1,Labios,1,3.12,24,4.17


La categoría sobre la que Sephora Collection dispone de un mayor número de productos exitosos es: Pinceles & accesorios. Además, con bastante diferencia con respecto al resto, pues casi el 60% de sus productos exitosos corresponden a la misma. Por el otro lado, las categorías con un menor número de productos exitosos son: Desmaquillante (2 productos) y Labios (1 producto).

- Categorías con mayor volumen de productos exitosos (% sobre el total exitosos SC): Pinceles & accesorios domina claramente con 19 productos exitosos, representando un 59.38% del total de productos exitosos dentro de Sephora Collection. Además, su catálogo total tiene 48 productos, lo que implica que esta categoría aporta más de la mitad de los éxitos de la marca. A distancia, las otras categorías:
    - Rostro con 6 productos (18.75%).
    - Ojos con 4 productos (12.50%).
    - Desmaquillante con 2 productos (6.25%).
    - Labios con solo 1 producto (3.12%).
- Categorías más eficientes internamente (% de éxito sobre total de productos en la categoría SC)
    - Desmaquillante es la categoría con mayor eficiencia interna: 2 productos exitosos de solo 3 productos totales, es decir, un 66.67% de éxito.
    - Le sigue Pinceles & accesorios con un 39.58% (19 de 48 productos).
    - Rostro tiene una tasa de éxito del 16.22%.
    - Ojos sólo un 7.55% de sus productos son exitosos, a pesar de ser una categoría con más productos (53).
    - Labios es la menos eficiente, con solo el 4.17%.

### Productos exitosos por subcategoría

In [564]:
total_sc_por_subcat = df[df['nombre_marca'] == 'Sephora Collection'].groupby('nombre_subcategoria').size().reset_index(name='total_productos_SC')
exitosos_sc_por_subcat = exitosos_espec.groupby('nombre_subcategoria').size().reset_index(name='count')
exitosos_sc_por_subcat['%/Total_exitosos_SC'] = ((exitosos_sc_por_subcat['count'] / len(exitosos_espec)) * 100).round(2)
exitosos_sc_por_subcat = exitosos_sc_por_subcat.merge(total_sc_por_subcat, on='nombre_subcategoria', how='left')
exitosos_sc_por_subcat['%_exitosos_sobre_subcat_SC'] = ((exitosos_sc_por_subcat['count'] / exitosos_sc_por_subcat['total_productos_SC']) * 100).round(2)
exitosos_sc_por_subcat = exitosos_sc_por_subcat.sort_values(by='%_exitosos_sobre_subcat_SC', ascending=False)
exitosos_sc_por_subcat.head()

,nombre_subcategoria,count,%/Total_exitosos_SC,total_productos_SC,%_exitosos_sobre_subcat_SC
3,Desmaquillante rostro,1,3.12,1,100.00
7,Leche desmaquillante,1,3.12,1,100.00
2,Contouring,2,6.25,3,66.67
4,Esponja de maquillaje,2,6.25,4,50.00
9,Polvos bronceadores,2,6.25,4,50.00


Como se observa, “Brocha rostro” domina absolutamente entre los éxitos, representando más de un tercio de todos los productos exitosos en Sephora Collection. Aunque solo tienen 1 producto, tanto Desmaquillante rostro como Leche desmaquillante muestran una eficiencia del 100%, lo que sugiere gran aceptación o muy buen rendimiento de esas unidades únicas.

In [517]:
mediana_precio_exitosos = exitosos_espec["precio"].median()
mediana_precio_exitosos

np.float64(18.49)

La mediana del precio de los productos exitosos de Sephora Collection está incluso por encima (18,49 euros) de la mediana del precio de los productos totales de la marca (13,99 euros), lo que sugiere que en esta marca concreta tampoco ocurre que precios más bajos en los productos sean la razón de mayor éxito en los productos. Incluso podríamos decir que precios un pelín más altos dentro de los rangos de la marca corresponden a productos muy bien valorados y que Sephora Collection se podría beneficiar de ello.

In [518]:
mediana_variaciones_exitosos = exitosos_espec["numero_variaciones"].median()
mediana_variaciones_exitosos

np.float64(0.0)

De igual forma parece que aunque Sephora Collection de por si tenga poca variedad en sus productos en cuanto a tonos o formatos, no es un problema, pues sus productos más exitosos podemos observar que mantienen el mismo perfil y por tanto un mayor número de variaciones en el producto no va a justificar un mayor éxito.

### Precio mediano por categorías

In [519]:
precio_mediano_categoria = (df_especifico.groupby("nombre_categoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_categoria.tail()

,nombre_categoria,precio
3,Labios,13.99
4,Ojos,13.99
0,Cejas,12.99
10,Uñas,10.99
9,Sacapuntas,3.99


### Precio mediano por subcategorías

In [520]:
precio_mediano_subcategoria = (df_especifico.groupby("nombre_subcategoria")["precio"].median().reset_index().sort_values("precio", ascending=False)).round(2)
precio_mediano_subcategoria.head()

,nombre_subcategoria,precio
27,Paleta de sombras de ojos,32.99
16,Estuches multi-usos,27.00
13,Estuche de pinceles,24.99
14,Estuches de labiales,22.00
15,Estuches de manicura,19.99


Dentro de las categorías y subcategorías donde Sephora Collection a día de hoy ya tiene presencia, cuenta con el precio mediano más alto destacable en una categoría concreta: Estuches maquillaje (24 euros) y dentro de las subcategorías aquellas 3 con precios medianos más altos son: Paleta de sombras de ojos (32,99), Estuches multi-usos (27), Estuche de pinceles (24,99). Por el otro extremo, la categoría también destacable por tener un precio mediano bastante menor al del resto de las categorías son los Sacapuntas (3,99) y dentro de las subcategorías encontraríamos: Limpiador pinceles y Quitaesmalte (8,99 euros ambas).

### Número de variaciones mediano por categorías

In [521]:
variaciones_median_cat = (df_especifico.groupby("nombre_categoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_cat.head()

,nombre_categoria,numero_variaciones
3,Labios,9.0
0,Cejas,4.5
8,Rostro,3.0
5,Paletas maquillaje,1.5
1,Desmaquillante,0.0


### Número de variaciones mediano por subcategorías

In [522]:
variaciones_median_subcat = (df_especifico.groupby("nombre_subcategoria")["numero_variaciones"].median().reset_index().sort_values("numero_variaciones", ascending=False)).round(2)
variaciones_median_subcat.head()

,nombre_subcategoria,numero_variaciones
4,Base de maquillaje,19.0
24,Lápiz de labios,17.0
3,Barra de labios líquida,15.0
2,Barra de labios,12.0
1,Antiojeras & corrector,11.0


En cuanto al número de variaciones por producto, Sephora Collection cuenta con un número de variaciones mediano igual a 0 en la mayoría de sus categorías excepto en Labios (9 variaciones), Cejas (4,5), Rostro (3) y Paletas de maquillaje (1,5). En cuanto a las subcategorías cuenta con 16 subcategorías con un número de variaciones mediano superior a 0, y dentro de estas, aquellas con un mayor número son las bases de maquillaje (19) y los lápices de ojos (17), y por el otro extremo, aquellas con un menor número son las paletas de rostro y los polvos bronceadores (1,5 variaciones cada una).

## Resumen Conclusiones Más Relevantes Análisis (posicionando a Sephora Collection)

### Del más General

- Cuenta con 1822 productos de maquillaje.
- Cuenta con 79 marcas, 13 categorías y 48 subcategorías.
- Valores representativos
    - Precio: 34,99
    - Número de valoraciones: 42,5
    - Valoración: 4,4
    - Número de variaciones: 2
- No se observa una relación directa entre el precio y el reconocimiento de los productos. Los productos más caros tienen buenas valoraciones, aunque un número menor de valoraciones. Los productos con mayor número de valoraciones y buenas reseñas no son necesariamente más baratos, sino que tienen un precio promedio. Por otro lado, los productos con menor éxito y visibilidad tampoco están relacionados con precios más altos, sino que también mantienen un precio promedio. La correlación calculada entre precio y éxito: 0,05. 
- Sephora Collection tiene la que más presencia (número de productos ofertados): 198 productos (10,87%)
- El Rostro, los Ojos y los Labios son las categorías con más presencia en Sephora. En cuanto a subcategorías son las Máscaras de pestañas, las Barras de labios y los Coloretes
- Productos exitosos en Sephora (valoración >= 4,5 y numero de valoraciones >= 50): 21,57%. Marcas como Natasha Denona, Huda Beauty y Anastasia Beverly Hills combinan alta eficiencia interna con volumen significativo, lo que las convierte en marcas estratégicas y consistentes. Otras como Glow Recipe y Pai tienen una eficiencia del 100%, pero con catálogos muy limitados, lo que reduce su peso real en el negocio. Sephora Collection, pese a tener un 16% de éxito interno, sigue siendo una de las marcas más relevantes por volumen absoluto, debido a su fuerte presencia en catálogo.
- Productos exitosos por categorías: Rostro es tanto la categoría más representada entre los éxitos totales como una de las más eficientes internamente (24.24%). Esto refuerza su papel central en la oferta de Sephora. Labios y Ojos tienen un volumen alto de productos exitosos, pero con eficiencias algo más bajas (20.12% y 16.94% respectivamente), lo que podría indicar una mayor dispersión en la calidad o el interés del cliente. Paletas de maquillaje sobresale como la categoría más eficaz en proporción (40.82%), aunque con un peso moderado entre los éxitos globales. Pinceles & accesorios combina alto volumen de éxito con buena eficiencia interna, lo que sugiere que, aunque no es la categoría más numerosa, sus productos tienden a funcionar muy bien. Cejas también muestra una alta eficiencia relativa, lo que la hace destacar como una categoría nicho pero efectiva.
- Productos exitosos por subcategorías: Colorete y Paleta de sombras de ojos son subcategorías que no solo aportan un volumen significativo de productos exitosos, sino que también mantienen una buena eficiencia interna (más del 38% de sus productos son exitosos). Brocha rostro y Sombras de ojos tienen buen peso dentro de los éxitos totales, pero su eficiencia interna es algo menor (29.13% y 30.26% respectivamente), lo que podría reflejar una oferta más amplia y variada con resultados más heterogéneos. Cremas BB & CC y Paleta rostro destacan por su altísima tasa de éxito interna (más del 45%), mostrando una concentración importante de productos bien valorados en catálogos relativamente pequeños. Estuches de pinceles, aunque no muy numerosos, también tienen un buen rendimiento relativo (36.36%), apoyando la idea de que ciertos productos específicos y de nicho logran un gran éxito.
- Sephora Collection se posiciona entre las 5 marcas más baratas (precio mediano de 13,99 euros)
- Las categorías con precios medianos más altos son las Paletas de maquillaje, el Rostro y los Ojos, mientras que las categorías con precios medianos más bajos son los Rizadores de pestañas, Uñas y Sacapuntas.
- Las subcategorías con precios medianos más altos son las Paletas de rostro, las Paletas de sombras de ojos y las Paletas de contouring. Por el lado contrario, las subcategorías con precios medianos más bajos son los Esmaltes de uñas, los Quitaesmaltes y los Limpiadores de pinceles. 
- Sephora Collection domina en número de productos ofertados en Sephora en las siguientes categorías (4/13): Estuches maquillaje, Labios, Ojos, Pinceles & accesorios. Y en las siguientes subcategorías (13/48): Brocha Rostro, Contouring, Esmalte de uñas, Estuche de pinceles, Estuches de labiales, Estuches de manicura, Estuches multi-usos, Eyeliner, Limpiador pinceles, Máscara de pestañas, Pestañas postizas, Pincel sombra de ojos, Sombras de ojos. 
- Sephora Collection se posiciona entre las marcas que como número mediano de variaciones cuenta con 0 (es decir, muy escasas opciones en la mayoría de sus productos).
- Dentro de las categorías, las únicas 3 que disponen de un número mediano de variaciones distinto de 0 son los Labios, Rostro y Cejas. Dentro de las subcategorías, las que disponen de un mayor número de variaciones mediano en sus productos son: Bases de maquillaje, Antiojeras & corrector, Crema de color, Gel de uñas.
- Outliers: 
    - Los productos más caros (outliers por precio) se concentran especialmente en la marca Sisley y la subcategoría de bases de maquillaje, aunque La Mer tiene una mediana de precio más alta de forma general.
    - Algunos productos con muchísimas valoraciones destacan especialmente (como uno de Anastasia Beverly Hills y otro de NARS), pero en general, las marcas Benefit y la subcategoría de máscara de pestañas son las más frecuentes entre los productos con muchas valoraciones.
    - Muchos productos (232) no han recibido aún valoraciones, siendo Dior y barras de labios los más comunes en este grupo.
    - Entre los peor valorados (valoración mínima ≥ 1), Sephora Collection aparece con frecuencia, lo que se explica porque es la marca con más productos en catálogo.
    - En número de variaciones, destacan dos productos de Rem Beauty, aunque no sea la marca con más variedad global. En general, bases de maquillaje y de nuevo Sephora Collection son las más frecuentes en este grupo de outliers.

### Del más Específico 

- Valores representativos: 
    - Precio: 13,99
    - Número de valoraciones: 26
    - Valoración: 4,4
    - Número de variaciones: 0
- El precio influye un poco más en el éxito de los productos pero sigue sin ser la causa principal. Correlación calculada entre precio y éxito: 0,31.
- Cuenta con 11 categorías y 40 subcategorías. No tiene presencia en las categorías de Papeles matificantes y Pestañas postizas. No tiene presencia en las subcategorías de Paleta multi-usos, Estuche para cejas, Estuches de ojos, Cremas BB & CC, Crema de color, Paleta de contouring, Gel uñas y Estuches de rostro.
- Las categorías donde cuenta con mayor % de productos son: Ojos, Pinceles & accesorios, Rostro y Labios. Donde cuenta con un menor % son: Desmaquillante, Paletas maquillaje, Rizador pestañas, Sacapuntas.
- Las subcategorías donde cuenta con mayor % de productos son: Brochas rostro, Sombras de ojos, Máscaras de pestañas. Donde cuenta con un menor % son: Base de sombra de ojos, Aceite demaquillante, Barra de labios líquida, Limpiador pinceles, Leche desmaquillante, Desmaquillante rostro, Voluminizador labios. 
- 32 productos exitosos. Suponen el 8,14% dentro de Sephora y el 16,16% dentro del total de Sephora Collection. 
- Productos exitosos por categorías: Pinceles & accesorios no solo aporta la mayoría de los productos exitosos, sino que también mantiene una tasa de éxito interna bastante alta (casi 40%), confirmando que esta categoría es un fuerte motor de éxito dentro de la marca. Desmaquillantes, aunque pequeño en número, tiene una eficiencia sobresaliente, lo que indica que casi todos los productos en esta categoría son bien recibidos. Las categorías de Rostro y Ojos tienen un peso moderado en el volumen de éxitos, pero su eficiencia interna sugiere que hay espacio para mejorar o que la competencia dentro de estas categorías es mayor. La categoría Labios parece ser un área con menor éxito relativo dentro de Sephora Collection, tanto en volumen como en tasa interna, lo que podría indicar menos enfoque o menor rendimiento comercial en esta categoría.
- Productos exitosos por subcategorías: Brochas de rostro son el pilar de los éxitos de Sephora Collection. No solo son la subcategoría más abundante entre los productos exitosos, sino que además tienen una excelente eficiencia interna (48% de sus productos son exitosos). Estuches de pinceles y sombras de ojos también destacan en volumen, pero tienen menor eficiencia que las brochas de rostro. Subcategorías pequeñas pero altamente exitosas como contouring, esponjas de maquillaje o los desmaquillantes, merecen atención estratégica: aunque su número de productos es bajo, tienen altas tasas de éxito. Esto sugiere que en Sephora Collection los productos de accesorios y herramientas (brochas, estuches, esponjas) no solo son numerosos entre los éxitos, sino también muy eficientes, reflejando una alta demanda o valoración positiva por parte de los clientes.
- Productos exitosos. Precio: 18,49 euros. Número de variaciones: 0. Un menor precio o mayores variaciones de producto no justifican mayor éxito en sus productos. 
- Categoría con precio mediano más alto: Estuches maquillaje, frente a la categoría con precio mediano más bajo: Sacapuntas. Por el lado de las subcategorías, aquellas con precios medianos más altos son las paletas de sombras de ojos, los estuches multi-usos y los estuches de pinceles, y aquellas con precios medianos más bajos son los limpiadores de pinceles y los quitaesmaltes.
- En cuanto al número de variaciones por producto, las categorías de Labios, Cejas, Rostro y Paletas de maquillaje son las únicas donde el número mediano es distinto de cero. Aquellas subcategorías con un número de variaciones mediano superior son las bases de maquillaje y los lápices de ojos, y aquellas con un menor número son las paletas de rostro y los polvos bronceadores.
- Outliers:
    - En precio, un estuche de pinceles destaca claramente como el más caro, aunque esta subcategoría no suele tener precios altos en general, lo que indica una alta dispersión dentro de ella.
    - En número de valoraciones, un eyeliner y un lápiz para cejas sobresalen tanto por su popularidad como por su buena valoración (4,3), además de tener un precio típico de la marca (13 €).
    - Solo 18 productos no han recibido valoraciones (mayoría estuches multiusos), y entre los peor valorados (solo 8 con nota baja), no se observa ningún patrón claro por subcategoría o precio.
    - En variaciones, destaca una base de maquillaje, en línea con la tendencia general de esta subcategoría a ofrecer múltiples tonos. Las bases son también las más frecuentes entre los outliers por variaciones.

En este análisis se ha estudiado la relación entre el precio de los productos y su éxito, medido en función de su valoración media y el número de valoraciones. Cabe señalar que los precios analizados corresponden al precio en bruto de cada producto, sin tener en cuenta su volumen o cantidad (por ejemplo, mililitros o gramos). Esto implica que, en algunos casos, productos que parecen más baratos podrían no serlo si se considera el precio por unidad de volumen, y viceversa. Un ejemplo claro son las colonias, donde los formatos más pequeños suelen tener un precio absoluto menor, pero un precio por mililitro mucho más elevado.

Esta limitación en el análisis también puede afectar a la identificación de valores atípicos (outliers). Algunos productos que actualmente aparecen como outliers en términos de precio podrían no serlo si se tuviera en cuenta su tamaño. Por ejemplo, un producto con un precio bruto muy alto podría estar justificado si se trata de un formato grande, lo cual no se refleja en el análisis actual.

Para futuros scrapeos, sería muy recomendable intentar extraer también el dato del volumen de cada producto. Esto permitiría calcular un precio ajustado (por ejemplo, €/ml), ofreciendo una perspectiva más precisa a la hora de estudiar la relación entre el precio y la percepción de valor o satisfacción del cliente.

En un dashboard esto mismo del análisis más específico para Sephora Collection podría hacerse para todas las marcas de forma muy sencilla y rápida con el uso de gráficos a partir del análisis más general y a través de emplear filtros.

### Ejemplo concreto de incoherencia y justificación del poco uso de determinados campos extraídos

En cuanto a las características más específicas de los productos —como los formatos, acabados, responsabilidades, texturas, entre otros—, he optado por no profundizar en su análisis general. Esta decisión se debe a que, tras revisar la página web (es decir, no es un error de scrapeo sino que viene de la propia web), se detectó una falta de coherencia en la asignación de estos atributos: en múltiples casos, los filtros se encontraban mezclados y la categorización de los productos no siempre respondía a criterios lógicos. Por tanto, al no considerarse campos completamente fiables, estos atributos no se emplearán en los análisis agregados. No obstante, podrían explorarse puntualmente en el estudio individual de productos concretos, como herramienta orientativa para identificar características potencialmente recomendables. Aunque no todos los productos están mal categorizados, existen casos aislados en los que se aprecia una ubicación poco coherente en cuanto a sus atributos.

In [523]:
query_incoherencia = """
SELECT 
    p.nombre,
    p.descripcion,
    s.nombre_subcategoria,
    t.nombre_textura
FROM productos p
LEFT JOIN subcategorias s ON p.id_subcategoria = s.id_subcategoria
LEFT JOIN producto_textura pt ON p.id_producto = pt.id_producto
LEFT JOIN texturas t ON pt.id_textura = t.id_textura
WHERE 
    p.descripcion = 'Polvos compactos formato de viaje' and t.nombre_textura = 'Liquido';
"""

In [524]:
df_incoherencia = ejecutar_query(query_incoherencia)
df_incoherencia

,nombre,descripcion,nombre_subcategoria,nombre_textura
0,Vanish Airbrush Pressed Powder,Polvos compactos formato de viaje,Polvos matificantes,Liquido


En este ejemplo pues es evidente que los polvos compactos no pertenecen por ejemplo a la textura de producto líquido

### Justificación en la decisión de emplear la mediana en vez de la media en todo el análisis

A lo largo del análisis, se ha optado por utilizar la mediana como medida de tendencia central para las variables numéricas (como precio, valoración o número de valoraciones) en lugar de la media. Esta elección se debe a que, tras una revisión exploratoria de los datos, se ha detectado la presencia de valores atípicos (outliers) que distorsionan significativamente la media y, por tanto, no representan con precisión el comportamiento general de los productos analizados. La mediana, al no verse afectada por estos extremos, proporciona una visión más robusta y representativa del comportamiento central de los datos, especialmente en distribuciones asimétricas o con alta dispersión. Por ello, se considera que la mediana refleja de manera más fiable la realidad del mercado en este contexto y permite tomar decisiones más fundamentadas.

## Explorar detalles sobre las subcategorías de los productos a lanzar

In [532]:
df_brochas_rostro = df[df["nombre_subcategoria"] == "Brocha rostro"]
df_brochas_rostro.describe(include='object').T

,count,unique,top,freq
url_producto,103,103,https://www.sephora.es/p/hoola-wavemaker-%e2%8...,1
nombre,102,95,Brocha profesional,4
descripcion,88,61,"Aplicación intuitiva, acabado perfecto",12
nombre_marca,103,20,Sephora Collection,25
nombre_categoria,103,1,Pinceles & accesorios,103
nombre_subcategoria,103,1,Brocha rostro,103


In [533]:
df_brochas_rostro_exito = df[(df["nombre_subcategoria"] == "Brocha rostro") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_brochas_rostro_exito.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,30.0,1100.633333,548.209089,110.00,710.250,1113.0,1646.00,1807.0
precio,30.0,38.329667,18.606989,11.99,24.990,36.5,46.75,78.0
numero_valoraciones,30.0,333.600000,370.718836,51.00,101.750,175.5,397.50,1649.0
valoracion,30.0,4.666667,0.134762,4.50,4.525,4.7,4.80,4.9
numero_variaciones,30.0,0.000000,0.000000,0.00,0.000,0.0,0.00,0.0
exito,30.0,1.000000,0.000000,1.00,1.000,1.0,1.00,1.0


In [534]:
df_brochas_rostro_sinexito = df[(df["nombre_subcategoria"] == "Brocha rostro") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_brochas_rostro_sinexito.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,73.0,987.561644,572.214072,22.00,425.00,765.0,1612.0,1815.0
precio,73.0,38.298904,18.459829,11.99,24.99,37.0,51.0,95.0
numero_valoraciones,73.0,36.821918,94.370037,0.00,1.00,8.0,26.0,543.0
valoracion,73.0,3.552055,1.864367,0.00,3.70,4.4,4.8,5.0
numero_variaciones,73.0,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0
exito,73.0,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0


In [535]:
df_brochas_rostro_espec = df[(df["nombre_subcategoria"] == "Brocha rostro") & (df["nombre_marca"] == "Sephora Collection")]
df_brochas_rostro_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,25.0,617.7600,397.686254,110.00,377.00,385.00,716.00,1570.00
precio,25.0,17.5904,6.318122,11.99,11.99,14.99,24.99,34.99
numero_valoraciones,25.0,182.9600,301.909683,2.00,20.00,60.00,171.00,1136.00
valoracion,25.0,4.7480,0.198158,4.10,4.70,4.80,4.90,5.00
numero_variaciones,25.0,0.0000,0.000000,0.00,0.00,0.00,0.00,0.00
exito,25.0,0.4800,0.509902,0.00,0.00,0.00,1.00,1.00


In [536]:
df_brochas_rostro_exito_espec = df[(df["nombre_subcategoria"] == "Brocha rostro") & (df["nombre_marca"] == "Sephora Collection") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_brochas_rostro_exito_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,12.0,549.833333,216.098392,110.00,372.50,709.000,714.500,724.00
precio,12.0,21.324167,6.664970,11.99,15.99,22.495,24.990,34.99
numero_valoraciones,12.0,325.083333,376.678657,60.00,102.50,143.000,367.250,1136.00
valoracion,12.0,4.758333,0.124011,4.50,4.70,4.800,4.825,4.90
numero_variaciones,12.0,0.000000,0.000000,0.00,0.00,0.000,0.000,0.00
exito,12.0,1.000000,0.000000,1.00,1.00,1.000,1.000,1.00


In [537]:
df_brochas_rostro_sinexito_espec = df[(df["nombre_subcategoria"] == "Brocha rostro") & (df["nombre_marca"] == "Sephora Collection") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_brochas_rostro_sinexito_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,13.0,680.461538,514.420647,370.00,380.00,383.00,719.00,1570.00
precio,13.0,14.143846,3.508232,11.99,11.99,12.99,13.99,24.99
numero_valoraciones,13.0,51.769231,115.722480,2.00,14.00,20.00,26.00,435.00
valoracion,13.0,4.738462,0.253438,4.10,4.70,4.80,4.90,5.00
numero_variaciones,13.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00
exito,13.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00


In [538]:
df_brochas_rostro_sin_espec = df[(df["nombre_subcategoria"] == "Brocha rostro") & (df["nombre_marca"] != "Sephora Collection")]
df_brochas_rostro_sin_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,78.0,1149.576923,551.408457,22.0,686.50,1262.50,1634.75,1815.0
precio,78.0,44.948077,15.892459,18.0,31.00,43.50,58.75,95.0
numero_valoraciones,78.0,104.128205,233.388999,0.0,1.25,12.00,88.50,1649.0
valoracion,78.0,3.597436,1.806192,0.0,3.70,4.45,4.70,5.0
numero_variaciones,78.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0
exito,78.0,0.230769,0.424052,0.0,0.00,0.00,0.00,1.0


In [539]:
df_brochas_rostro_sinespec = df[(df["nombre_subcategoria"] == "Brocha rostro") & (df["nombre_marca"] != "Sephora Collection") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_brochas_rostro_sinespec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,18.0,1467.833333,354.315122,649.0,1252.25,1615.5,1727.5,1807.0
precio,18.0,49.666667,14.887816,24.0,38.50,46.0,63.0,78.0
numero_valoraciones,18.0,339.277778,377.556822,51.0,104.25,205.0,420.5,1649.0
valoracion,18.0,4.605556,0.105564,4.5,4.50,4.6,4.7,4.8
numero_variaciones,18.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0
exito,18.0,1.000000,0.000000,1.0,1.00,1.0,1.0,1.0


In [540]:
df_brochas_rostro_sinexito_sinespec = df[(df["nombre_subcategoria"] == "Brocha rostro") & (df["nombre_marca"] != "Sephora Collection") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_brochas_rostro_sinexito_sinespec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,60.0,1054.100000,566.099701,22.0,578.0,1027.500,1626.500,1815.0
precio,60.0,43.532500,16.029056,18.0,29.0,40.495,55.500,95.0
numero_valoraciones,60.0,33.583333,89.913007,0.0,1.0,5.000,25.500,543.0
valoracion,60.0,3.295000,1.962481,0.0,1.9,4.200,4.725,5.0
numero_variaciones,60.0,0.000000,0.000000,0.0,0.0,0.000,0.000,0.0
exito,60.0,0.000000,0.000000,0.0,0.0,0.000,0.000,0.0


--------------------------------------------------------------

In [541]:
df_cremas_bbycc = df[df["nombre_subcategoria"] == "Cremas BB & CC"]
df_cremas_bbycc.describe(include='object').T

,count,unique,top,freq
url_producto,18,18,https://www.sephora.es/p/bb-burst---gel-crema-...,1
nombre,18,16,CC Red Correct,2
descripcion,18,17,Tratamiento iluminador corrector de rojeces,2
nombre_marca,18,10,Erborian,8
nombre_categoria,18,1,Rostro,18
nombre_subcategoria,18,1,Cremas BB & CC,18


In [542]:
df_cremas_bbycc_exito = df[(df["nombre_subcategoria"] == "Cremas BB & CC") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_cremas_bbycc_exito.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,9.0,1155.777778,485.944899,181.0,944.0,986.0,1595.0,1701.0
precio,9.0,35.222222,10.268615,24.0,26.0,32.0,45.0,48.0
numero_valoraciones,9.0,370.000000,332.709333,67.0,86.0,341.0,455.0,1070.0
valoracion,9.0,4.633333,0.100000,4.5,4.6,4.6,4.7,4.8
numero_variaciones,9.0,2.333333,1.658312,0.0,2.0,2.0,3.0,5.0
exito,9.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [543]:
df_cremas_bbycc_sinexito = df[(df["nombre_subcategoria"] == "Cremas BB & CC") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_cremas_bbycc_sinexito.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,9.0,939.555556,619.891747,117.0,404.0,810.0,1501.0,1691.0
precio,9.0,45.555556,26.476929,26.0,37.0,39.0,43.0,114.0
numero_valoraciones,9.0,38.222222,46.770123,0.0,1.0,19.0,49.0,124.0
valoracion,9.0,3.300000,1.946150,0.0,3.2,4.0,4.5,5.0
numero_variaciones,9.0,5.111111,7.339013,0.0,2.0,3.0,4.0,24.0
exito,9.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [547]:
df_cremas_bbycc_sin_espec = df[(df["nombre_subcategoria"] == "Cremas BB & CC") & (df["nombre_marca"] != "Sephora Collection")]
df_cremas_bbycc_sin_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,18.0,1047.666667,551.663260,117.0,668.250,985.50,1571.50,1701.0
precio,18.0,40.388889,20.193589,24.0,26.250,39.00,44.75,114.0
numero_valoraciones,18.0,204.111111,286.808788,0.0,26.250,78.00,309.25,1070.0
valoracion,18.0,3.966667,1.502547,0.0,4.025,4.55,4.70,5.0
numero_variaciones,18.0,3.722222,5.355652,0.0,2.000,3.00,4.00,24.0
exito,18.0,0.500000,0.514496,0.0,0.000,0.50,1.00,1.0


In [548]:
df_cremas_bbycc_sinespec = df[(df["nombre_subcategoria"] == "Cremas BB & CC") & (df["nombre_marca"] != "Sephora Collection") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_cremas_bbycc_sinespec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,9.0,1155.777778,485.944899,181.0,944.0,986.0,1595.0,1701.0
precio,9.0,35.222222,10.268615,24.0,26.0,32.0,45.0,48.0
numero_valoraciones,9.0,370.000000,332.709333,67.0,86.0,341.0,455.0,1070.0
valoracion,9.0,4.633333,0.100000,4.5,4.6,4.6,4.7,4.8
numero_variaciones,9.0,2.333333,1.658312,0.0,2.0,2.0,3.0,5.0
exito,9.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [549]:
df_cremas_bbycc_sinexito_sinespec = df[(df["nombre_subcategoria"] == "Cremas BB & CC") & (df["nombre_marca"] != "Sephora Collection") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_cremas_bbycc_sinexito_sinespec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,9.0,939.555556,619.891747,117.0,404.0,810.0,1501.0,1691.0
precio,9.0,45.555556,26.476929,26.0,37.0,39.0,43.0,114.0
numero_valoraciones,9.0,38.222222,46.770123,0.0,1.0,19.0,49.0,124.0
valoracion,9.0,3.300000,1.946150,0.0,3.2,4.0,4.5,5.0
numero_variaciones,9.0,5.111111,7.339013,0.0,2.0,3.0,4.0,24.0
exito,9.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


------------------------------------------------------------------------------------------

In [377]:
df_sombras_ojos = df[df["nombre_subcategoria"] == "Sombras de ojos"]
df_sombras_ojos.describe(include='object').T

,count,unique,top,freq
url_producto,76,76,https://www.sephora.es/p/starlet---sombra-de-o...,1
nombre,76,73,Colorful Sombra de Ojos,3
descripcion,73,48,Paleta de sombras de ojos,12
nombre_marca,76,28,Sephora Collection,17
nombre_categoria,76,1,Ojos,76
nombre_subcategoria,76,1,Sombras de ojos,76


In [384]:
df_sombras_ojos_exito = df[(df["nombre_subcategoria"] == "Sombras de ojos") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_sombras_ojos_exito.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,23.0,811.043478,412.700252,248.0,568.0,721.0,982.00,1614.00
precio,23.0,46.735652,29.401031,10.0,28.5,37.0,67.99,139.99
numero_valoraciones,23.0,355.608696,359.070104,53.0,145.5,223.0,456.50,1682.00
valoracion,23.0,4.700000,0.120605,4.5,4.7,4.7,4.75,4.90
numero_variaciones,23.0,4.652174,5.951384,0.0,0.0,3.0,6.50,21.00
exito,23.0,1.000000,0.000000,1.0,1.0,1.0,1.00,1.00


In [386]:
df_sombras_ojos_sinexito = df[(df["nombre_subcategoria"] == "Sombras de ojos") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_sombras_ojos_sinexito.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,53.0,720.754717,476.607196,64.00,365.00,693.0,983.0,1768.0
precio,53.0,36.733774,24.079661,7.99,19.99,34.0,43.0,144.0
numero_valoraciones,53.0,54.358491,190.500565,0.00,2.00,9.0,29.0,1314.0
valoracion,53.0,3.396226,1.859276,0.00,3.40,4.3,4.6,5.0
numero_variaciones,53.0,4.698113,5.648638,0.00,0.00,3.0,7.0,30.0
exito,53.0,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0


In [387]:
df_sombras_ojos_espec = df[(df["nombre_subcategoria"] == "Sombras de ojos") & (df["nombre_marca"] == "Sephora Collection")]
df_sombras_ojos_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,17.0,435.058824,313.733659,105.00,239.0,441.00,506.0,1350.00
precio,17.0,12.700588,3.994816,7.99,8.0,13.99,14.0,19.99
numero_valoraciones,17.0,43.647059,64.299049,0.00,2.0,8.00,29.0,189.00
valoracion,17.0,3.705882,1.652449,0.00,3.6,4.50,4.7,5.00
numero_variaciones,17.0,5.705882,7.278090,0.00,3.0,4.00,6.0,30.00
exito,17.0,0.235294,0.437237,0.00,0.0,0.00,0.0,1.00


In [393]:
df_sombras_ojos_exito_espec = df[(df["nombre_subcategoria"] == "Sombras de ojos") & (df["nombre_marca"] == "Sephora Collection") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_sombras_ojos_exito_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,4.0,463.500,249.137847,248.0,251.0000,442.500,655.0000,721.00
precio,4.0,14.995,4.160132,10.0,12.9925,14.995,16.9975,19.99
numero_valoraciones,4.0,153.000,26.394444,126.0,140.2500,148.500,161.2500,189.00
valoracion,4.0,4.700,0.000000,4.7,4.7000,4.700,4.7000,4.70
numero_variaciones,4.0,3.250,2.500000,0.0,2.2500,3.500,4.5000,6.00
exito,4.0,1.000,0.000000,1.0,1.0000,1.000,1.0000,1.00


In [395]:
df_sombras_ojos_sinexito_espec = df[(df["nombre_subcategoria"] == "Sombras de ojos") & (df["nombre_marca"] == "Sephora Collection") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_sombras_ojos_sinexito_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,13.0,426.307692,339.659237,105.00,204.0,441.0,505.00,1350.00
precio,13.0,11.994615,3.828406,7.99,8.0,12.0,13.99,19.99
numero_valoraciones,13.0,10.000000,11.217546,0.00,2.0,5.0,15.00,29.00
valoracion,13.0,3.400000,1.791647,0.00,3.5,3.7,5.00,5.00
numero_variaciones,13.0,6.461538,8.150822,0.00,3.0,4.0,9.00,30.00
exito,13.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.00


In [382]:
df_sombras_ojos_sin_espec = df[(df["nombre_subcategoria"] == "Sombras de ojos") & (df["nombre_marca"] != "Sephora Collection")]
df_sombras_ojos_sin_espec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,59.0,838.271186,454.277859,64.0,479.50,753.0,1179.0,1768.0
precio,59.0,47.557627,24.382956,18.0,33.00,40.0,59.0,144.0
numero_valoraciones,59.0,174.881356,318.557150,0.0,5.50,31.0,200.5,1682.0
valoracion,59.0,3.815254,1.678958,0.0,4.05,4.5,4.7,5.0
numero_variaciones,59.0,4.389831,5.199526,0.0,0.00,2.0,7.0,21.0
exito,59.0,0.322034,0.471267,0.0,0.00,0.0,1.0,1.0


In [416]:
df_sombras_ojos_exito_sinespec = df[(df["nombre_subcategoria"] == "Sombras de ojos") & (df["nombre_marca"] != "Sephora Collection") & (df["numero_valoraciones"] >= 50) & (df["valoracion"] >= 4.5)]
df_sombras_ojos_exito_sinespec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,19.0,884.210526,406.612234,273.0,619.00,768.0,1185.00,1614.00
precio,19.0,53.417895,27.975249,27.0,34.00,37.0,73.99,139.99
numero_valoraciones,19.0,398.263158,382.653723,53.0,148.00,287.0,465.00,1682.00
valoracion,19.0,4.700000,0.133333,4.5,4.65,4.7,4.80,4.90
numero_variaciones,19.0,4.947368,6.459048,0.0,0.00,0.0,7.50,21.00
exito,19.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.00


In [417]:
df_sombras_ojos_sinexito_sinespec = df[(df["nombre_subcategoria"] == "Sombras de ojos") & (df["nombre_marca"] != "Sephora Collection") & ((df["numero_valoraciones"] < 50) | (df["valoracion"] < 4.5))]
df_sombras_ojos_sinexito_sinespec.describe().T

,count,mean,std,min,25%,50%,75%,max
id_producto,40.0,816.450,478.625647,64.0,460.7500,747.0,1156.500,1768.0
precio,40.0,44.774,22.322311,18.0,32.4925,40.0,52.000,144.0
numero_valoraciones,40.0,68.775,217.897940,0.0,1.0000,12.5,32.250,1314.0
valoracion,40.0,3.395,1.903027,0.0,3.3000,4.4,4.525,5.0
numero_variaciones,40.0,4.125,4.552753,0.0,0.0000,2.5,7.000,19.0
exito,40.0,0.000,0.000000,0.0,0.0000,0.0,0.000,0.0


In [406]:
df[(df["nombre_marca"] == 'Natasha Denona') & (df["nombre_subcategoria"] == 'Sombras de ojos')].describe().T.loc[["precio", "numero_variaciones"]]

,count,mean,std,min,25%,50%,75%,max
precio,7.0,65.848571,17.748615,27.0,67.99,73.99,74.49,74.99
numero_variaciones,7.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.00


### Columna que refleja de forma numérica el éxito

In [531]:
df["exito"] = ((df["valoracion"] >= 4.5) & (df["numero_valoraciones"] >= 50)).astype(int)
df.head()

,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones,exito
0,https://www.sephora.es/p/soft-lit-naturally-lu...,1,Soft' Lit Naturally Luminous Foundation,Base de maquillaje hidratante,Fenty Beauty,Rostro,Base de maquillaje,40.00,73,4.7,50,1
1,https://www.sephora.es/p/triclone-skin-tech-fo...,2,Triclone Skin Tech Foundation,Cobertura media con arnica fermentada,Haus Labs,Rostro,Base de maquillaje,49.99,35,4.5,51,0
2,https://www.sephora.es/p/triclone-skin-tech-hy...,3,Triclone Skin Tech Hydrating Concealer with Fe...,Corrector,Haus Labs,Rostro,Antiojeras & corrector,34.99,20,4.1,30,0
3,https://www.sephora.es/p/color-fuse-talc-free-...,4,Color Fuse Talc-Free Powder Blush With Ferment...,Colorete en polvo,Haus Labs,Rostro,Colorete,32.99,18,4.8,6,0
4,https://www.sephora.es/p/bio-radiant-gel-powde...,5,Bio-Radiant Gel-Powder Highlighter,Iluminador con arnica fermentada,Haus Labs,Rostro,Iluminador,46.99,8,4.8,5,0


In [526]:
print(df[["precio", "exito"]].corr())

          precio     exito
precio  1.000000  0.049574
exito   0.049574  1.000000


In [530]:
df_especifico["exito"] = ((df_especifico["valoracion"] >= 4.5) & (df_especifico["numero_valoraciones"] >= 50)).astype(int)
df_especifico.head()

C:\Users\raque\AppData\Local\Temp\ipykernel_20540\2885498066.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,url_producto,id_producto,nombre,descripcion,nombre_marca,nombre_categoria,nombre_subcategoria,precio,numero_valoraciones,valoracion,numero_variaciones,exito
11,https://www.sephora.es/p/el-estuche-de-brochas...,12,El Estuche de Brochas de Iniciación,"Aplicación intuitiva, acabado perfecto",Sephora Collection,Pinceles & accesorios,Estuche de pinceles,42.99,94,4.9,0,1
48,https://www.sephora.es/p/get-unready*---estuch...,49,Get (Un)Ready*,Estuche trío belleza,Sephora Collection,Estuches maquillaje,Estuches multi-usos,24.00,0,0.0,0,0
51,https://www.sephora.es/p/time-to-glitter---pur...,52,Time To Glitter,Purpurina Instantánea 2 geles de purpurina,Sephora Collection,None,None,18.00,0,0.0,0,0
59,https://www.sephora.es/p/size-up---m%c3%a1scar...,60,Size Up,Máscara Volumen Extralargo Inmediato (Formato ...,Sephora Collection,Ojos,Máscara de pestañas,7.00,640,3.9,0,0
60,https://www.sephora.es/p/colorful-luminizer---...,61,Colorful Luminizer,Polvos iluminadores para el rostro,Sephora Collection,Rostro,Iluminador,12.99,0,0.0,3,0


In [529]:
print(df_especifico[["precio", "exito"]].corr())

          precio     exito
precio  1.000000  0.312166
exito   0.312166  1.000000


En general, el precio no muestra una relación significativa con el éxito de los productos en Sephora, ya que la correlación observada es prácticamente nula. Esto sugiere que el precio, por sí solo, no es un factor determinante del éxito.

Sin embargo, al analizar específicamente la línea Sephora Collection, se encuentra una correlación positiva (r = 0.31) entre precio y éxito. Aunque esta relación sigue siendo débil, podría indicar que, dentro de una marca conocida por su enfoque económico, los productos ligeramente más caros son percibidos como de mayor calidad o valor diferencial, lo que puede influir positivamente en su recepción.

Aun así, dado que la correlación es moderada y no muy fuerte, se puede concluir que el precio no es el principal factor de éxito, y que otros elementos —como la calidad percibida, la reputación, las reseñas o el marketing— probablemente tienen un mayor peso en la valoración final por parte de los consumidores.

Lo que hay que tener en cuenta entonces es que Sephora Collection es la marca más presente en número de productos dentro de Sephora (el 10,87% de los productos de Sephora corresponden a Sephora Collection), que todavía le quedan cubrir 8/48 subcategorías pero que en 13/48 es la marca que domina en número de productos. Además, que se caracteriza por sus precios competitivos (una de las marcas más baratas de Sephora) y que de forma general está bastante bien valorada. De hecho, el 8,14% (32 productos) de los productos exitosos de Sephora corresponden a Sephora Collection, cifra donde únicamente le supera la marca de Benefit Cosmetics con el 8,91%. De forma general, se observa Sephora Collection como una buena marca en Sephora que dispone de buena relación calidad-precios y que a ella acudirán aquellos clientes que quieran productos más económicos y que ocasionalmente podrán acudir a productos también muy buenos pero más caros de otras marcas que también dispone de ellas Sephora.

Resulta clave que cualquier lanzamiento siga reforzando su propuesta de valor: precio competitivo, buena relación calidad-precio y atractivo para un consumidor que busca rendimiento sin lujo innecesario.

En la decisión sobre el próximo lanzamiento de producto de Sephora se pueden dar 3 enfoques: 

LANZAMIENTO SEGURO: BROCHA DE ROSTRO 

La elección de una brocha de rostro como producto para un lanzamiento seguro es una decisión altamente respaldada por los datos. 
- Liderazgo interno de la subcategoría: Es la subcategoría donde Sephora Collection tiene mayor presencia de catálogo: 25 productos, lo que representa el 12,63% de toda la marca. También es la que mayor número de productos exitosos aporta a la marca: 12 de 32 productos exitosos, es decir, el 37,5% del total.
- Alta eficiencia de éxito dentro de la marca: Casi la mitad de las brochas de rostro (48%) de Sephora Collection son exitosas, lo que muestra que este tipo de producto tiene una alta tasa de conversión a éxito dentro de la propia marca.
- Buen posicionamiento dentro de Sephora en general: Las brochas de rostro son la segunda subcategoría con más productos exitosos en Sephora (7,63% del total de éxitos), lo que confirma que es una categoría bien valorada por el consumidor en general. De esos 30 productos exitosos en esta subcategoría, 12 pertenecen a Sephora Collection, lo que consolida aún más su fuerza competitiva.
- Modelo de producto sencillo y con margen de maniobra en precio:
    - Las brochas de rostro suelen tener variaciones nulas (mediana = 0), lo que reduce la complejidad del desarrollo y facilita el lanzamiento de un producto único.
    - El rango de precios dentro de Sephora Collection en esta categoría va de 11 a 34,99 €, mientras que en otras marcas de Sephora el rango es más alto (18 a 95 €).
    - Se observa que el precio mediano de las brochas exitosas en Sephora Collection es significativamente mayor que el de las no exitosas (22,5 € vs. 12,99 €), lo que indica que el cliente está dispuesto a pagar más si la calidad lo justifica, incluso dentro del marco económico de la marca.

Aunque todos los indicadores apuntan a que este sería un lanzamiento seguro, sí puede haber cierto riesgo de saturación debido a que: Sephora Collection ya tiene 25 brochas de rostro en su línea. La subcategoría en general está muy explotada tanto por la marca como por otras marcas competidoras en Sephora. A nivel visual o funcional, puede resultar difícil destacar sin aportar un elemento diferencial claro.

Para minimizar el riesgo de saturación y maximizar las probabilidades de éxito, se recomienda optar por una brocha de rostro con una propuesta diferenciadora, que mantenga la seguridad de la categoría pero incorpore un enfoque creativo:
- Diseño con doble funcionalidad: por ejemplo, una brocha dual para contorno y difuminado, o base + iluminador.
- Edición especial con diseño coleccionable o temático: manteniendo precios dentro del rango alto de Sephora Collection (22–28 €), con valor percibido más alto.
- Colaboración limitada con una artista gráfica, diseñadora o embajadora digital de la marca.
- Mejora del packaging o formato ecológico/sostenible, un plus cada vez más valorado por el consumidor.
Este tipo de enfoque permitiría mantener los beneficios de lanzar dentro de una subcategoría probadamente exitosa, pero aportando suficiente innovación visual o funcional para evitar el estancamiento en ventas o la canibalización interna.

LANZAMIENTO ARRIESGADO: CREMA BB & CC

Apostar por entrar en una nueva subcategoría: BB & CC Creams. Si bien se trata de una decisión más arriesgada al no contar con precedentes internos, los datos sugieren que es una apuesta con alto potencial si se ejecuta con inteligencia estratégica.

A pesar de ser una subcategoría aún no explorada por Sephora Collection, presenta varios indicadores que mitigan el riesgo percibido:
- Subcategoría vacante para Sephora Collection
    - Sephora Collection no tiene aún presencia en BB & CC Creams, por lo que un lanzamiento en este campo no generaría canibalización interna.
    - Entrar en esta subcategoría permitiría a la marca ampliar cobertura y continuar su crecimiento desde una base ya consolidada en otras categorías.
- Alto índice de éxito dentro de la subcategoría
    - Aunque solo hay 18 productos totales en esta categoría en Sephora, 9 de ellos son considerados exitosos, lo que supone un índice del 50% de éxito.
    - Esto convierte a las BB & CC Creams en la subcategoría sin presencia actual de Sephora Collection con mayor proporción de éxito, representando un 2,29% del total de productos exitosos de Sephora.
- Buen rendimiento general de la categoría
    - Las BB & CC Creams muestran una mediana de valoraciones de 78, con una valoración media altísima de 4,55/5, lo cual evidencia que los productos, incluso los que no están clasificados como “exitosos”, siguen siendo muy bien valorados por los clientes.
    - Esto sugiere una categoría saludable, donde el consumidor tiene ya una alta predisposición a confiar y comprar este tipo de productos.
- Dominancia de una marca de mayor precio
    - La marca que lidera esta subcategoría es Erborian, con 8 productos, de los cuales 6 son exitosos.
    - Erborian tiene un precio mediano de 30 €, lo que la posiciona en un rango superior al habitual de Sephora Collection.
    - Esto abre una clara oportunidad para competir con una propuesta más accesible, sin sacrificar calidad, y atraer a un público que quiere una BB/CC Cream eficaz pero económica.

Analizando las diferencias entre productos exitosos y no exitosos en esta categoría:
- Productos exitosos tienen un precio mediano de 32 € y 2 variaciones.
- Productos no exitosos tienen un precio más alto (mediana de 39 €) y 3 variaciones.
Esto indica que más variaciones no implican más éxito y que, dentro de esta subcategoría, el precio sí puede ser un factor crítico para diferenciarse.

Por ello, el enfoque ideal para Sephora Collection sería:
- Un solo producto (sin variaciones) para minimizar la inversión inicial y validar la respuesta del mercado.
- Precio por debajo del mediano de los productos exitosos (idealmente 24–27 €), lo cual:
    - Mantendría la percepción de Sephora Collection como marca accesible.
    - Reforzaría su ventaja frente a Erborian y marcas similares.

Riesgos y como mitigarlos: 
- Desconocimiento del consumidor sobre Sephora Collection en productos de rostro con color y tratamiento
→ Solución: acompañar el lanzamiento con una fuerte estrategia de comunicación pedagógica sobre los beneficios del producto y su comparativa frente a los líderes actuales.
- Competencia fuerte y bien posicionada (Erborian, Clinique, etc.)
→ Solución: diferenciarse en precio y fórmula simplificada; Sephora Collection puede destacarse como la opción más sencilla y efectiva para uso diario.
- Mayor complejidad en percepción del producto vs. brochas
→ Solución: mantener fórmula lo más "universal" posible (para todo tipo de pieles, sin subtonos, fácil de aplicar) y recoger feedback del cliente desde el lanzamiento para decidir si expandir la gama con más tonos o acabados.

Dado que esta categoría se caracteriza por fórmulas efectivas, bien valoradas y con un alto índice de éxito, la clave no está en la creatividad disruptiva, sino en ofrecer una propuesta sencilla, clara y eficaz. Una BB o CC Cream con fórmula universal, sin complicaciones, enfocada en resultados visibles (hidratación, uniformidad y luminosidad), con buena tolerancia para todo tipo de pieles y un precio accesible, puede posicionarse como la alternativa diaria ideal frente a opciones más costosas del mercado.

LANZAMIENTO DE MEJORA: SOMBRAS DE OJOS 

Sephora Collection cuenta con una presencia consolidada en la subcategoría de sombras de ojos, con 17 productos actualmente en catálogo, de los cuales 4 son considerados exitosos. Esto representa una tasa de éxito del 23,53% dentro de la subcategoría y supone el 12,5% del total de productos exitosos de la marca. Además, Sephora Collection es la marca que más sombras de ojos vende dentro de Sephora, lo que refuerza su relevancia en esta subcategoría estratégica.

Sin embargo, la subcategoría muestra un amplio margen de mejora. En el contexto general de Sephora, las sombras de ojos son la quinta subcategoría con mayor proporción de productos exitosos (30,26%, con 23 de 76 productos). Aunque Sephora Collection lidera en número de productos, la mayoría de los productos exitosos en esta subcategoría pertenecen a otras marcas, concretamente a otras 27 marcas más (19 de 23).

Los productos sin éxito de Sephora Collection presentan una mediana de solo 5 valoraciones y una puntuación mediana de 3,7. En comparación, los productos no exitosos del resto de marcas alcanzan una mediana de 12,5 valoraciones y una valoración mediana de 4,4. Esta diferencia evidencia una mejor percepción general hacia otras marcas, incluso cuando sus productos no alcanzan el umbral de éxito. Por tanto, Sephora Collection tiene un gran potencial de mejora.

En términos de precio, Sephora Collection se posiciona claramente como una opción accesible: sus sombras de ojos tienen un precio mediano de 13,99 €, frente a los 40 € de mediana en el resto de marcas. Además, presenta una mediana de 4 variaciones por producto, también superior a la mediana de 2 que ofrecen otras marcas. Sin embargo, esta estrategia de accesibilidad y variedad no se ha traducido necesariamente en un mayor éxito comercial.

Analizando el rendimiento del resto de marcas, observamos que los productos exitosos presentan un precio mediano ligeramente inferior al de los productos sin éxito (37 € frente a 40 €), lo que sugiere que, si bien el precio puede influir en el éxito, lo hace de forma moderada. En cuanto a las variaciones, los productos exitosos tienen una mediana de 0, frente a 2,5 en los no exitosos, lo que indica que un mayor número de tonos no implica mejores resultados —incluso puede resultar contraproducente—.

En el caso concreto de Sephora Collection, esta tendencia se mantiene: los productos exitosos tienen un precio mediano de 14,99 €, mientras que los productos sin éxito se sitúan en los 12 €. Es decir, dentro del propio surtido de la marca, las sombras algo más caras están funcionando mejor, lo que refuerza la idea de que el menor precio por sí solo no garantiza éxito. La diferencia no es elevada, pero sí significativa: podría justificarse una ligera subida de precio si se acompaña de mejoras claras en la calidad percibida, la fórmula o el diseño.

Respecto al número de variaciones, las sombras de ojos exitosas de Sephora Collection tienen una mediana de 3,5, frente a 4 en los productos sin éxito, lo que refuerza la conclusión de que más variaciones no se traducen en mayor éxito. De hecho, ambas cifras superan ampliamente la mediana del resto de marcas, lo que sugiere que Sephora Collection podría beneficiarse de una estrategia más focalizada: reducir el número de variaciones a un surtido compacto pero muy bien seleccionado, y redirigir los esfuerzos hacia la mejora de la calidad del producto. Apostar por una mayor calidad a un precio aún competitivo parece ser el verdadero camino hacia el éxito en esta subcategoría.

En este contexto, Natasha Denona se posiciona como el principal competidor, con 6 productos exitosos en la categoría y un precio mediano de 73,99 €. Esto abre una oportunidad para que Sephora Collection se establezca como la alternativa de calidad a un precio muy inferior, con una propuesta simplificada, eficaz y accesible.

Un análisis del contenido de los productos también ofrece insights clave. Entre los productos exitosos del resto de las marcas, la descripción más frecuente es "paleta de sombras de ojos" (8 de 19), mientras que entre los productos no exitosos predomina "sombra de ojos" (6 de 40). Esto sugiere que el consumidor percibe un mayor valor en las paletas, probablemente por su versatilidad, variedad cromática y mejor relación cantidad-precio, lo que se traduce en una experiencia de uso más completa. Apostar por una paleta compacta pero bien curada puede, por tanto, ser una estrategia ganadora para Sephora Collection.
Resulta especialmente interesante que la descripción más repetida entre los productos exitosos sea "paleta de sombras de ojos", a pesar de que existe una subcategoría específica para este tipo de producto. Esto sugiere que algunas paletas son encontradas por los clientes tanto en la categoría general de sombras de ojos como en la subcategoría específica de paletas, lo cual amplifica su visibilidad. A día de hoy, Sephora Collection únicamente ofrece 3 productos en la subcategoría de Paletas de sombras de ojos, y ninguno de ellos es considerado exitoso. Esta escasa presencia contrasta con el buen rendimiento general de esta subcategoría dentro del catálogo de Sephora: se venden un total de 74 paletas, de las cuales el 39,19% son productos exitosos. Esta tasa de éxito es superior a la media de la categoría general de sombras de ojos (30,26%), lo que refuerza aún más la oportunidad de crecimiento.
En este contexto, ampliar y optimizar la presencia de Sephora Collection en la subcategoría de paletas —con una propuesta mejor formulada, bien posicionada y visiblemente diferenciada— representa una vía clara para aumentar el rendimiento y alinearse con las preferencias del consumidor actual.

A partir del análisis, se propone orientar el lanzamiento de mejora con las siguientes directrices:
- Apostar por una paleta compacta y sin variaciones de formato: Una sola referencia, sin múltiples versiones, que simplifique la oferta y potencie su impacto.
- Reducir el número de tonos incluidos: Surtido pequeño (3–5 colores), bien curado y con combinaciones versátiles para uso diario o de ocasión.
- Subir ligeramente el precio si mejora la calidad: Posicionar el producto entre 15–17 €, manteniendo el ADN accesible, pero reforzando el valor percibido.
- Enfocarse en la mejora de la calidad del producto: Revisar fórmula, pigmentación, duración y facilidad de uso para competir en percepción con marcas premium.
- Optimizar el diseño y el packaging: Diseño atractivo, moderno y funcional, que refleje calidad sin aumentar excesivamente los costes.
- Estrategia de visibilidad: reforzar el lanzamiento con acciones de marketing digital, activaciones con influencers, bundles, y campañas que impulsen reseñas y prueba del producto.

En resumen, la categoría de sombras de ojos representa una oportunidad clara de mejora para Sephora Collection. Aprovechar su posicionamiento como marca económica, pero elevar la calidad percibida y la estrategia de activación, puede transformar su desempeño en una de las subcategorías más competitivas y visibles de Sephora.